In [ ]:
#IMPORT LIBRARIES
import numpy as np
import pandas as pd
import fnmatch
import math
import time

import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline
#import missingno as msno

from sklearn.model_selection import cross_val_score, RandomizedSearchCV, GroupShuffleSplit
from sklearn.model_selection import GroupKFold, GridSearchCV
from sklearn import linear_model, metrics
from sklearn.svm import SVR, LinearSVR
from xgboost import XGBRegressor
from scipy.stats import uniform, randint

from sklearn.linear_model import LassoCV
from sklearn.linear_model import LassoLarsCV

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer

# DKT FS Loop with Lasso-LLars

In [ ]:
%%capture --no-stderr capture

for j in range(3,5): # MODIFY to (1,5) for all 4 datasets
    for z in range(1,4): # MODIFY to (1,4) for all 3 variables
        for q in range(1,21):
            data = j
            variable = z
            estimator = 1
            imaging = q
            #####################
            # DATASET SELECTION #
            #####################

            if data == 1 and variable == 1:
                dataset_name = "CLDRC_DropNA"
                model_var = "READ"
                df = pd.read_csv("df_vars_readfs_dropna_cldrc.csv", na_values = 'NaN', index_col=0)
            elif data == 1 and variable == 2:
                dataset_name = "CLDRC_DropNA"
                model_var = "IN"
                df = pd.read_csv("df_vars_infs_dropna_cldrc.csv", na_values = 'NaN', index_col=0)
            elif data == 1 and variable == 3:
                dataset_name = "CLDRC_DropNA"
                model_var = "HI"
                df = pd.read_csv("df_vars_hifs_dropna_cldrc.csv", na_values = 'NaN', index_col=0)
            elif data == 2 and variable == 1:
                dataset_name = "CLDRC_Imputed"
                model_var = "READ"
                df = pd.read_csv("df_vars_readfs_imputed_cldrc.csv", index_col=0)
            elif data == 2 and variable == 2:
                dataset_name = "CLDRC_Imputed"
                model_var = "IN"
                df = pd.read_csv("df_vars_infs_imputed_cldrc.csv", index_col=0)
            elif data == 2 and variable == 3:
                dataset_name = "CLDRC_Imputed"
                model_var = "HI"
                df = pd.read_csv("df_vars_hifs_imputed_cldrc.csv", index_col=0)
            elif data == 3 and variable == 1:
                dataset_name = "Hub_DropNA"
                model_var = "READ"
                df = pd.read_csv("df_vars_readfs_dropna_hub.csv", na_values = 'NaN', index_col=0)
            elif data == 3 and variable == 2:
                dataset_name = "Hub_DropNA"
                model_var = "IN"
                df = pd.read_csv("df_vars_infs_dropna_hub.csv", na_values = 'NaN', index_col=0)
            elif data == 3 and variable == 3:
                dataset_name = "Hub_DropNA"
                model_var = "HI"
                df = pd.read_csv("df_vars_hifs_dropna_hub.csv", na_values = 'NaN', index_col=0)
            elif data == 4 and variable == 1:
                dataset_name = "Hub_Imputed"
                model_var = "READ"
                df = pd.read_csv("df_vars_readfs_imputed_hub.csv", index_col=0)
            elif data == 4 and variable == 2:
                dataset_name = "Hub_Imputed"
                model_var = "IN"
                df = pd.read_csv("df_vars_infs_imputed_hub.csv", index_col=0)
            elif data == 4 and variable == 3:
                dataset_name = "Hub_Imputed"
                model_var = "HI"
                df = pd.read_csv("df_vars_hifs_imputed_hub.csv", index_col=0)
            else:
                print("Data and Variable Not Selected")

            #######################
            # IMPORT IMAGING DATA #
            #######################

            if imaging == 1:
                df_i = pd.read_csv('1.DKTarea_ICVresid_hub.csv'); i_filename = 'DKTarea_ICVresid_hub.csv' #1
            elif imaging == 2:
                df_i = pd.read_csv('1.DKTarea_SA_hub.csv'); i_filename = 'DKTarea_SA_hub.csv' #2
            elif imaging == 3:
                df_i = pd.read_csv('1.DKTarea_SAresid_hub.csv'); i_filename = 'DKTarea_SAresid_hub.csv' #3
            elif imaging == 4:
                df_i = pd.read_csv('1.DKTarea_asym_hub.csv'); i_filename = 'DKTarea_asym_hub.csv'
            elif imaging == 5:
                df_i = pd.read_csv('1.DKTarea_hub.csv'); i_filename = 'DKTarea_hub.csv'
            elif imaging == 6:
                df_i = pd.read_csv('1.DKTaseg_ICVresid_hub.csv'); i_filename = 'DKTaseg_ICVresid_hub.csv'
            elif imaging == 7:
                df_i = pd.read_csv('1.DKTaseg_hub.csv'); i_filename = 'DKTaseg_hub.csv' #4
            elif imaging == 8:
                df_i = pd.read_csv('1.DKTlgi_ICVresid_hub.csv'); i_filename = 'DKTlgi_ICVresid_hub.csv'
            elif imaging == 9:
                df_i = pd.read_csv('1.DKTlgi_SAresid_hub.csv'); i_filename = 'DKTlgi_SAresid_hub.csv'
            elif imaging == 10:
                df_i = pd.read_csv('1.DKTlgi_asym_hub.csv'); i_filename = 'DKTlgi_asym_hub.csv'
            elif imaging == 11:
                df_i = pd.read_csv('1.DKTlgi_hub.csv'); i_filename = 'DKTlgi_hub.csv'
            elif imaging == 12:
                df_i = pd.read_csv('1.DKTthick_ICVresid_hub.csv'); i_filename = 'DKTthick_ICVresid_hub.csv' #5
            elif imaging == 13:
                df_i = pd.read_csv('1.DKTthick_MT_hub.csv'); i_filename = 'DKTthick_MT_hub.csv' #6
            elif imaging == 14:
                df_i = pd.read_csv('1.DKTthick_MTresid_hub.csv'); i_filename = 'DKTthick_MTresid_hub.csv' #7
            elif imaging == 15:
                df_i = pd.read_csv('1.DKTthick_asym_hub.csv'); i_filename = 'DKTthick_asym_hub.csv'
            elif imaging == 16:
                df_i = pd.read_csv('1.DKTthick_hub.csv'); i_filename = 'DKTthick_hub.csv'
            elif imaging == 17:
                df_i = pd.read_csv('1.DKTvol_ICVresid_hub.csv'); i_filename = 'DKTvol_ICVresid_hub.csv'
            elif imaging == 18:
                df_i = pd.read_csv('1.DKTvol_SAresid_hub.csv'); i_filename = 'DKTvol_SAresid_hub.csv'
            elif imaging == 19:
                df_i = pd.read_csv('1.DKTvol_asym_hub.csv'); i_filename = 'DKTvol_asym_hub.csv'
            elif imaging == 20:
                df_i = pd.read_csv('1.DKTvol_hub.csv'); i_filename = 'DKTvol_hub.csv'
            else:
                i_filename = 'No_Imaging_in_Model'

            ################################
            # COMBINE BEHAVIORAL & IMAGING #
            ################################

            if data == 3 or data == 4:
                df_with_i = df.merge(df_i, on='scanid1')
                df_with_i = df_with_i.dropna()
            #    df_with_i.drop('Unnamed: 0_y', axis=1, inplace=True)
            #    df.rename(columns={'Unnamed: 0_x':'Unnamed: 0'}, inplace=True)
            #    df_with_i.drop(df_with_i.columns[13], axis=1, inplace=True)
            #    df_with_i.drop('Unnamed: 0_y', axis=1, inplace=True)
            #    df_with_i.reset_index(drop=True,inplace=True)

            else:
                i_filename = 'No_Imaging_in_Model'
            print("Imaging File: ", i_filename)
            print("Dataset Name: ", dataset_name)
            print("Model Variable: ", model_var)

            if 'Unnamed: 0' in df.columns:
                print('Unnamed: 0 in df')
            if 'Unnamed: 0' in df_i.columns:
                print('Unnamed: 0 in df_i')
            if 'Unnamed: 0' in df_with_i.columns:
                print('Unnamed: 0 in df_with_i')

            ######################
            # SETTING ORIG_INDEX #
            ######################

            df.reset_index(inplace=True)
            df_i.reset_index(inplace=True)
            df_with_i.reset_index(inplace=True)

            df.rename(columns={'index':'orig_index'}, inplace=True)
            df_i.rename(columns={'index':'orig_index'}, inplace=True)
            df_with_i.rename(columns={'index':'orig_index'}, inplace=True)

            #######################
            # ESTIMATOR SELECTION #
            #######################

            if estimator == 1:
                model_estimator = "Linear_SVR"
                param_dist = {"C": uniform(.0001,5),
                              "epsilon": uniform(0,1)}
                alg = LinearSVR(max_iter=10000)
            elif estimator == 2:
                model_estimator = "SVR_Linear_Kernel"
                param_dist = {"C": uniform(.0001,5),
                              "epsilon": uniform(0,1)}
                alg = SVR(kernel = 'linear')
            elif estimator == 3:
                model_estimator = "SVR_Nonlinear_RBF"
                param_dist = {"C": uniform(.0001,5),
                             "epsilon": uniform(0,1)}
                alg = SVR(kernel = 'rbf', gamma = 'auto')
            elif estimator == 4:
                model_estimator="XGBoost"
                param_dist = {"n_estimators": randint(5,100),
                              "learning_rate": uniform(0.01,0.5),
                              "gamma": uniform(0.0,0.5),
                              "max_depth": randint(3,10),
                              "min_child_weight": randint(1,8),
                              "subsample": uniform(.6,0.4),
                              "colsample_bytree": uniform(.3,0.7),
                              "reg_lambda": uniform(0.001,5),
                              "reg_alpha": uniform(0.00001,1)}
                alg = XGBRegressor(n_jobs=6)
            elif estimator == 5:
                model_estimator = "Linear_Regression"
                alg = linear_model.LinearRegression(n_jobs=8)
            else:
                print("Estimator Not Selected")

            #print("Model Estimator: ", model_estimator)

            #########################
            # DATASET PREPROCESSING #
            #########################

            baselength = len(df.columns)
            base = [3,4]
            base2 = list(range(6,len(df.columns)))
            base.extend(base2)
            full1_preprocessor = make_column_transformer(
                (StandardScaler(), base),
                remainder='passthrough')

            df.rename(columns={'Unnamed: 0':'orig_index'}, inplace=True)
            ###################################
            ### POSSIBLY SHOULD BE EXCLUDED ###
            ###################################
            if data == 3 or data == 4:
                df_with_i.rename(columns={'Unnamed: 0':'orig_index'}, inplace=True)
            ###################################
            ### POSSIBLY SHOULD BE EXCLUDED ###
            ###################################

            i_index1 = index1 = df.columns.values[3]
            index2 = df.columns.values[6:].tolist()
            i_index3 = index3 = df.columns.values[5]
            i_index4 = index4 = df.columns.values[0:3].tolist()
            i_index5 = index5 = df.columns.values[4]

            index2.extend(index4)
            index2.insert(0,index1)
            index2.insert(1,index5)
            index2.extend([index3])

            df_scaled = full1_preprocessor.fit_transform(df)
            df_scaled = pd.DataFrame(data=df_scaled)
            df_scaled.columns = index2

            if data == 3 or data == 4:
                base3 = list(range(6,len(df_i.columns)+baselength-2))

                full = [3,4]
                full.extend(base3)
                full2_preprocessor = make_column_transformer(
                    (StandardScaler(), full),
                    remainder='passthrough')

                i_index2 = df_with_i.columns.values[6:].tolist()
                i_index2.extend(i_index4)
                i_index2.insert(0,i_index1)
                i_index2.insert(1,i_index5)
                i_index2.extend([i_index3])

                df_scaled_i = full2_preprocessor.fit_transform(df_with_i)
                df_scaled_i = pd.DataFrame(data=df_scaled_i)
                df_scaled_i.columns = i_index2

            print("Data Preprocessed - Scaler Applied")

            ######################
            # FEATURES SELECTION #
            ######################

            x1 = [1,2]
            x2 = [len(df_scaled.columns)-1]
            x3 = list(range(3,len(df_scaled.columns)-4))
            X1_vars = x1 + x2
            X2_vars = x1 + x2 + x3

            if data == 3 or data ==4:
                x5 = [len(df_scaled_i.columns)-1]
                x4 = list(range(len(df_scaled.columns)-4,len(df_scaled_i.columns)-4))
                X3_vars = x1 + x5 + x3 + x4

            X1 = df_scaled.iloc[:,X1_vars]
            y1 = df_scaled.iloc[:,0]
            if data == 1 or data == 2:
                groups1 = groups2 = df_scaled['famid']
            elif data == 3 or data == 4:
                groups1 = groups2 = df_scaled['fid']
            else:
                print('Dataset not set')

            X2 = df_scaled.iloc[:,X2_vars]
            y2 = df_scaled.iloc[:,0]
            features_full1 = ['Age']
            for i in [X2_vars[1:]]:
                features_full1.extend(df_scaled.columns[i])
            features_demo = features_full1[0:3]

            if data == 3 or data == 4:
                X3 = df_scaled_i.iloc[:,X3_vars]
                y3 = df_scaled_i.iloc[:,0]
                groups3 = df_scaled_i['fid'] # CHANGED FROM df_scaled.columns TO GET CORRECT # FAMID/GROUPS
                features_full2 = ['Age']
                for i in [X3_vars[1:]]:
                    features_full2.extend(df_scaled_i.columns[i])


            ##############################
            # CROSS-VALIDATION SELECTION #
            ##############################

            inner_rstate = 9052017
            outer_rstate = 12301985
            #inner_rstate = np.random.RandomState(9052017)
            #outer_rstate = np.random.RandomState(12301985)

            cv=GroupShuffleSplit
            #cv=GroupKFold
            cv_inner = cv(n_splits=20, test_size=0.2, random_state=inner_rstate)
            cv_outer = cv(n_splits=10, test_size=0.2, random_state=outer_rstate)
            n_iter=200
            n_iter_str = str(n_iter)

            print("Features Selected")
            print("Inner Training CV: ", cv_inner)
            print("Outer Testing CV: ", cv_outer)

            #print("Dataset: ", dataset_name)
            #print("Model Variable: ", model_var)
            #print("Model Estimator: ", model_estimator)
            #print("Imaging File: ", i_filename)

            ############################
            # SET SPECIFICITY OUTCOMES #
            ############################

            if dataset_name == "CLDRC_DropNA" and model_var == "READ":
                specificity_file = "PARTIAL CLDRC CROSS-CONSTRUCT READ SPECIFICITY RESULTS (N=1265)"
                df_outcomes = pd.read_csv("cldrc_1265_specificity_outcomes.csv")
            elif dataset_name == "CLDRC_DropNA" and (model_var == "IN" or model_var == "HI"):
                specificity_file = "PARTIAL CLDRC CROSS-CONSTRUCT IN/HI SPECIFICITY RESULTS (N=1190)"
                df_outcomes = pd.read_csv("cldrc_1190_specificity_outcomes.csv")
            elif dataset_name == "CLDRC_Imputed" and (model_var == "READ" or model_var == "IN" or model_var == "HI"):
                specificity_file = "FULL CLDRC CROSS-CONSTRUCT SPECIFICITY RESULTS (N=1675)"
                df_outcomes = pd.read_csv("cldrc_1675_specificity_outcomes.csv")
            elif dataset_name == "Hub_DropNA" and model_var == "READ":
                specificity_file = "PARTIAL HUB CROSS-CONSTRUCT READ SPECIFICITY RESULTS (N=214)"
                df_outcomes = pd.read_csv("hub_214_specificity_outcomes.csv")
            elif dataset_name == "Hub_DropNA" and (model_var == "IN" or model_var == "HI"):
                specificity_file = "PATIAL HUB CROSS-CONSTRUCT IN/HI SPECIFICITY RESULTS (N=209)"
                df_outcomes = pd.read_csv("hub_209_specificity_outcomes.csv")
            elif dataset_name == "Hub_Imputed" and (model_var == "READ" or model_var == "IN" or model_var == "HI"):
                specificity_file = "FULL HUB CROSS-CONSTRUCT SPECIFICITY RESULTS (N=324)"
                df_outcomes = pd.read_csv("hub_324_specificity_outcomes.csv")
            else:
                print("Dataset and Model Variables Not Set")

            print("")
            print("Specificity Filename: ", specificity_file)
            print("")
            r2 = df_outcomes['out_read_fs']
            i2 = df_outcomes['out_in_fs']
            h2 = df_outcomes['out_hi_fs']
            m2 = df_outcomes['out_math_fs']

            ###############################
            ### NESTED CROSS-VALIDATORS ###
            ###############################

            from sklearn.model_selection import ShuffleSplit

            cvreg=ShuffleSplit
            cvreg_inner = cvreg(n_splits=20, test_size=0.2, random_state=123085)
            cvreg_outer = cvreg(n_splits=9, test_size=0.2, random_state=90517)

            if (data == 3 or data == 4):
                #######################
                ### LassoCV Imaging ###
                #######################

                print("LASSO_CV IMAGING")
                search_time_start_lassoi = time.time()
                alphas_lassocvi = np.logspace(-4, -0.3, 100)
                lasso_cvi = LassoCV(alphas=alphas_lassocvi, cv=cvreg_inner, max_iter=100000, tol=0.0001, n_jobs=4)
                lasso_cvi_score = []

                for k, (trainCV, testCV) in enumerate(cvreg_outer.split(X3, y3, groups=groups3)):
                    lasso_cvi.fit(X3.values[trainCV], y3[trainCV])
                    print("[fold {0}] alpha: {1:.5f}, score: {2:.5f}".
                          format(k, lasso_cvi.alpha_, lasso_cvi.score(X3.values[testCV], y3[testCV])))
                    lasso_cvi_score.append(lasso_cvi.score(X3.values[testCV], y3[testCV]))
                search_time_stop_lassoi = time.time()

                feature_list_lassoi = []
                for i in list(range(0,len(X3.columns))):
                    feature_list_lassoi.append([lasso_cvi.coef_[i], X3.columns[i]])
                    df_featimp_lassoi = pd.DataFrame(data=feature_list_lassoi, columns=['coef','feature'])
                df_featimp_lassoi.reindex(df_featimp_lassoi.coef.abs().sort_values(ascending=False).index)
                df_featimp_sum_lassoi = df_featimp_lassoi[(df_featimp_lassoi['coef'] > .001) | (df_featimp_lassoi['coef'] < -.001)]
                print(df_featimp_sum_lassoi.reindex(df_featimp_sum_lassoi.coef.abs().sort_values(ascending=False).index))


                ###########################
                ### LassoLarsCV Imaging ###
                ###########################

                print("LASSOLARS_CV IMAGING")
                search_time_start_llarsi = time.time()
                llars_cvi = LassoLarsCV(cv=cvreg_inner, max_iter=10000, n_jobs=4)
                llars_cvi_score = []

                for k, (trainCV, testCV) in enumerate(cvreg_outer.split(X3, y3, groups=groups3)):
                    llars_cvi.fit(X3.values[trainCV], y3[trainCV])
                    print("[fold {0}] alpha: {1:.5f}, score: {2:.5f}".
                          format(k, llars_cvi.alpha_, llars_cvi.score(X3.values[testCV], y3[testCV])))
                    llars_cvi_score.append(llars_cvi.score(X3.values[testCV], y3[testCV]))
                search_time_stop_llarsi = time.time()

                feature_list_lli = []
                for i in list(range(0,len(X3.columns))):
                    feature_list_lli.append([llars_cvi.coef_[i], X3.columns[i]])
                    df_featimp_lli = pd.DataFrame(data=feature_list_lli, columns=['coef','feature'])
                df_featimp_lli.reindex(df_featimp_lli.coef.abs().sort_values(ascending=False).index)
                df_featimp_sum_lli = df_featimp_lli[(df_featimp_lli['coef'] > .001) | (df_featimp_lli['coef'] < -.001)]
                print(df_featimp_sum_lli.reindex(df_featimp_sum_lli.coef.abs().sort_values(ascending=False).index))
            else:
                print("Imaging regularization not available for CLDRC dataset")

            print('')
            print("LassoCVI Mean: {0:.5f}; SD: {1:.5f}; alpha: {2:.5f}; time: {3:.3f}".
                    format(np.mean(lasso_cvi_score), np.std(lasso_cvi_score), lasso_cvi.alpha_, search_time_stop_lassoi-search_time_start_lassoi))
            print("LLarsCVI Mean: {0:.5f}; SD: {1:.5f}; alpha: {2:.5f}; time: {3:.3f}".
                    format(np.mean(llars_cvi_score), np.std(llars_cvi_score), llars_cvi.alpha_, search_time_stop_llarsi-search_time_start_llarsi))
            print('')
            print('Lasso Imaging Weights')
            print(df_featimp_sum_lassoi.reindex(df_featimp_sum_lassoi.coef.abs().sort_values(ascending=False).index))
            print('')
            print('LassoLars Imaging Weights')
            print(df_featimp_sum_lli.reindex(df_featimp_sum_lli.coef.abs().sort_values(ascending=False).index))
            print('')
            print('')
            print('')
            print('')
            print('')
            print('')

In [ ]:
with open("0.results_hubDKTvarloop_lassollars.txt", 'w') as f:
    f.write(capture.stdout)

# JUST IMAGING - DKT + FS5/TBSS Loop with Lasso-LLars

In [ ]:
%%capture --no-stderr capture

for j in range(4,5): # MODIFY to (1,5) for all 4 datasets
    for z in range(1,2): # MODIFY to (1,4) for all 3 variables
        for q in range(12,28): # MODIFY to (1,28) for all 27 imaging datasets
            data = j
            variable = z
            estimator = 1
            imaging = q
            #####################
            # DATASET SELECTION #
            #####################

            if data == 1 and variable == 1:
                dataset_name = "CLDRC_DropNA"
                model_var = "READ"
                df = pd.read_csv("df_vars_readfs_dropna_cldrc.csv", na_values = 'NaN', index_col=0)
            elif data == 1 and variable == 2:
                dataset_name = "CLDRC_DropNA"
                model_var = "IN"
                df = pd.read_csv("df_vars_infs_dropna_cldrc.csv", na_values = 'NaN', index_col=0)
            elif data == 1 and variable == 3:
                dataset_name = "CLDRC_DropNA"
                model_var = "HI"
                df = pd.read_csv("df_vars_hifs_dropna_cldrc.csv", na_values = 'NaN', index_col=0)
            elif data == 2 and variable == 1:
                dataset_name = "CLDRC_Imputed"
                model_var = "READ"
                df = pd.read_csv("df_vars_readfs_imputed_cldrc.csv", index_col=0)
            elif data == 2 and variable == 2:
                dataset_name = "CLDRC_Imputed"
                model_var = "IN"
                df = pd.read_csv("df_vars_infs_imputed_cldrc.csv", index_col=0)
            elif data == 2 and variable == 3:
                dataset_name = "CLDRC_Imputed"
                model_var = "HI"
                df = pd.read_csv("df_vars_hifs_imputed_cldrc.csv", index_col=0)
            elif data == 3 and variable == 1:
                dataset_name = "Hub_DropNA"
                model_var = "READ"
                df = pd.read_csv("df_vars_readfs_dropna_hub.csv", na_values = 'NaN', index_col=0)
            elif data == 3 and variable == 2:
                dataset_name = "Hub_DropNA"
                model_var = "IN"
                df = pd.read_csv("df_vars_infs_dropna_hub.csv", na_values = 'NaN', index_col=0)
            elif data == 3 and variable == 3:
                dataset_name = "Hub_DropNA"
                model_var = "HI"
                df = pd.read_csv("df_vars_hifs_dropna_hub.csv", na_values = 'NaN', index_col=0)
            elif data == 4 and variable == 1:
                dataset_name = "Hub_Imputed"
                model_var = "READ"
                df = pd.read_csv("df_vars_readfs_imputed_hub.csv", index_col=0)
            elif data == 4 and variable == 2:
                dataset_name = "Hub_Imputed"
                model_var = "IN"
                df = pd.read_csv("df_vars_infs_imputed_hub.csv", index_col=0)
            elif data == 4 and variable == 3:
                dataset_name = "Hub_Imputed"
                model_var = "HI"
                df = pd.read_csv("df_vars_hifs_imputed_hub.csv", index_col=0)
            else:
                print("Data and Variable Not Selected")

            #######################
            # IMPORT IMAGING DATA #
            #######################

            if imaging == 1:
                df_i = pd.read_csv('1.DKTarea_ICVresid_hub.csv'); i_filename = 'DKTarea_ICVresid_hub.csv' #1
            elif imaging == 2:
                df_i = pd.read_csv('1.DKTarea_SA_hub.csv'); i_filename = 'DKTarea_SA_hub.csv' #2
            elif imaging == 3:
                df_i = pd.read_csv('1.DKTarea_SAresid_hub.csv'); i_filename = 'DKTarea_SAresid_hub.csv' #3
            elif imaging == 4:
                df_i = pd.read_csv('1.DKTarea_asym_hub.csv'); i_filename = 'DKTarea_asym_hub.csv'
            elif imaging == 5:
                df_i = pd.read_csv('1.DKTarea_hub.csv'); i_filename = 'DKTarea_hub.csv'
            elif imaging == 6:
                df_i = pd.read_csv('1.DKTaseg_ICVresid_hub.csv'); i_filename = 'DKTaseg_ICVresid_hub.csv'
            elif imaging == 7:
                df_i = pd.read_csv('1.DKTaseg_hub.csv'); i_filename = 'DKTaseg_hub.csv' #4
            elif imaging == 8:
                df_i = pd.read_csv('1.DKTlgi_ICVresid_hub.csv'); i_filename = 'DKTlgi_ICVresid_hub.csv'
            elif imaging == 9:
                df_i = pd.read_csv('1.DKTlgi_SAresid_hub.csv'); i_filename = 'DKTlgi_SAresid_hub.csv'
            elif imaging == 10:
                df_i = pd.read_csv('1.DKTlgi_asym_hub.csv'); i_filename = 'DKTlgi_asym_hub.csv'
            elif imaging == 11:
                df_i = pd.read_csv('1.DKTlgi_hub.csv'); i_filename = 'DKTlgi_hub.csv'
            elif imaging == 12:
                df_i = pd.read_csv('1.DKTthick_ICVresid_hub.csv'); i_filename = 'DKTthick_ICVresid_hub.csv' #5
            elif imaging == 13:
                df_i = pd.read_csv('1.DKTthick_MT_hub.csv'); i_filename = 'DKTthick_MT_hub.csv' #6
            elif imaging == 14:
                df_i = pd.read_csv('1.DKTthick_MTresid_hub.csv'); i_filename = 'DKTthick_MTresid_hub.csv' #7
            elif imaging == 15:
                df_i = pd.read_csv('1.DKTthick_asym_hub.csv'); i_filename = 'DKTthick_asym_hub.csv'
            elif imaging == 16:
                df_i = pd.read_csv('1.DKTthick_hub.csv'); i_filename = 'DKTthick_hub.csv'
            elif imaging == 17:
                df_i = pd.read_csv('1.DKTvol_ICVresid_hub.csv'); i_filename = 'DKTvol_ICVresid_hub.csv'
            elif imaging == 18:
                df_i = pd.read_csv('1.DKTvol_SAresid_hub.csv'); i_filename = 'DKTvol_SAresid_hub.csv'
            elif imaging == 19:
                df_i = pd.read_csv('1.DKTvol_asym_hub.csv'); i_filename = 'DKTvol_asym_hub.csv'
            elif imaging == 20:
                df_i = pd.read_csv('1.DKTvol_hub.csv'); i_filename = 'DKTvol_hub.csv'
            elif imaging == 21:
                df_i = pd.read_csv('1.fs5new.csv', na_values='NaN', index_col=0); i_filename = "fs5_labels"
            elif imaging == 22:
                df_i = pd.read_csv('1.tbssall.csv', na_values='NaN', index_col=0); i_filename = "tbss"
            elif imaging == 23:
                df_i = pd.read_csv('1.aseg.csv', na_values='NaN', index_col=0); i_filename = "aseg"
            elif imaging == 24:
                df_i = pd.read_csv('1.fs5new_tbssall.csv', na_values='NaN', index_col=0); i_filename = "fs5_tbss"
            elif imaging == 25:
                df_i = pd.read_csv('1.fs5new_aseg.csv', na_values='NaN', index_col=0); i_filename = "fs5_aseg"
            elif imaging == 26:
                df_i = pd.read_csv('1.tbssall_aseg.csv', na_values='NaN', index_col=0); i_filename = "tbss_aseg"
            elif imaging == 27:
                df_i = pd.read_csv('1.fs5new_tbssall_aseg.csv', na_values='NaN', index_col=0); i_filename = "fs5_tbss_aseg"
            else:
                i_filename = 'No_Imaging_in_Model'

            ################################
            # COMBINE BEHAVIORAL & IMAGING #
            ################################

            if data == 3 or data == 4:
                df_with_i = df.merge(df_i, on='scanid1')
                df_with_i = df_with_i.dropna()
            #    df_with_i.drop('Unnamed: 0_y', axis=1, inplace=True)
            #    df.rename(columns={'Unnamed: 0_x':'Unnamed: 0'}, inplace=True)
            #    df_with_i.drop(df_with_i.columns[13], axis=1, inplace=True)
            #    df_with_i.drop('Unnamed: 0_y', axis=1, inplace=True)
            #    df_with_i.reset_index(drop=True,inplace=True)

            else:
                i_filename = 'No_Imaging_in_Model'
            print("Imaging File: ", i_filename)
            print("Dataset Name: ", dataset_name)
            print("Model Variable: ", model_var)

            if 'Unnamed: 0' in df.columns:
                print('Unnamed: 0 in df')
            if 'Unnamed: 0' in df_i.columns:
                print('Unnamed: 0 in df_i')
            if 'Unnamed: 0' in df_with_i.columns:
                print('Unnamed: 0 in df_with_i')

            ######################
            # SETTING ORIG_INDEX #
            ######################

            df.reset_index(inplace=True)
            df_i.reset_index(inplace=True)
            df_with_i.reset_index(inplace=True)

            df.rename(columns={'index':'orig_index'}, inplace=True)
            df_i.rename(columns={'index':'orig_index'}, inplace=True)
            df_with_i.rename(columns={'index':'orig_index'}, inplace=True)

            #######################
            # ESTIMATOR SELECTION #
            #######################

            if estimator == 1:
                model_estimator = "Linear_SVR"
                param_dist = {"C": uniform(.0001,5),
                              "epsilon": uniform(0,1)}
                alg = LinearSVR(max_iter=10000)
            elif estimator == 2:
                model_estimator = "SVR_Linear_Kernel"
                param_dist = {"C": uniform(.0001,5),
                              "epsilon": uniform(0,1)}
                alg = SVR(kernel = 'linear')
            elif estimator == 3:
                model_estimator = "SVR_Nonlinear_RBF"
                param_dist = {"C": uniform(.0001,5),
                             "epsilon": uniform(0,1)}
                alg = SVR(kernel = 'rbf', gamma = 'auto')
            elif estimator == 4:
                model_estimator="XGBoost"
                param_dist = {"n_estimators": randint(5,100),
                              "learning_rate": uniform(0.01,0.5),
                              "gamma": uniform(0.0,0.5),
                              "max_depth": randint(3,10),
                              "min_child_weight": randint(1,8),
                              "subsample": uniform(.6,0.4),
                              "colsample_bytree": uniform(.3,0.7),
                              "reg_lambda": uniform(0.001,5),
                              "reg_alpha": uniform(0.00001,1)}
                alg = XGBRegressor(n_jobs=6)
            elif estimator == 5:
                model_estimator = "Linear_Regression"
                alg = linear_model.LinearRegression(n_jobs=8)
            else:
                print("Estimator Not Selected")

            #print("Model Estimator: ", model_estimator)

            #########################
            # DATASET PREPROCESSING #
            #########################

            baselength = len(df.columns)
            base = [3,4]
            base2 = list(range(6,len(df.columns)))
            base.extend(base2)
            full1_preprocessor = make_column_transformer(
                (StandardScaler(), base),
                remainder='passthrough')

            df.rename(columns={'Unnamed: 0':'orig_index'}, inplace=True)
            ###################################
            ### POSSIBLY SHOULD BE EXCLUDED ###
            ###################################
            if data == 3 or data == 4:
                df_with_i.rename(columns={'Unnamed: 0':'orig_index'}, inplace=True)
            ###################################
            ### POSSIBLY SHOULD BE EXCLUDED ###
            ###################################

            i_index1 = index1 = df.columns.values[3]
            index2 = df.columns.values[6:].tolist()
            i_index3 = index3 = df.columns.values[5]
            i_index4 = index4 = df.columns.values[0:3].tolist()
            i_index5 = index5 = df.columns.values[4]

            index2.extend(index4)
            index2.insert(0,index1)
            index2.insert(1,index5)
            index2.extend([index3])

            df_scaled = full1_preprocessor.fit_transform(df)
            df_scaled = pd.DataFrame(data=df_scaled)
            df_scaled.columns = index2

            if data == 3 or data == 4:
                base3 = list(range(6,len(df_i.columns)+baselength-2))

                full = [3,4]
                full.extend(base3)
                full2_preprocessor = make_column_transformer(
                    (StandardScaler(), full),
                    remainder='passthrough')

                i_index2 = df_with_i.columns.values[6:].tolist()
                i_index2.extend(i_index4)
                i_index2.insert(0,i_index1)
                i_index2.insert(1,i_index5)
                i_index2.extend([i_index3])

                df_scaled_i = full2_preprocessor.fit_transform(df_with_i)
                df_scaled_i = pd.DataFrame(data=df_scaled_i)
                df_scaled_i.columns = i_index2

            print("Data Preprocessed - Scaler Applied")

            ######################
            # FEATURES SELECTION #    ####### READ: X3.columns[9:] ; IN & HI: X3.columns[11:]
            ######################

            x1 = [1,2]
            x2 = [len(df_scaled.columns)-1]
            x3 = list(range(3,len(df_scaled.columns)-4))
            X1_vars = x1 + x2
            X2_vars = x1 + x2 + x3

            if data == 3 or data ==4:
                x5 = [len(df_scaled_i.columns)-1]
                x4 = list(range(len(df_scaled.columns)-4,len(df_scaled_i.columns)-4))
                X3_vars = x1 + x5 + x3 + x4

            X1 = df_scaled.iloc[:,X1_vars]
            y1 = df_scaled.iloc[:,0]
            if data == 1 or data == 2:
                groups1 = groups2 = df_scaled['famid']
            elif data == 3 or data == 4:
                groups1 = groups2 = df_scaled['fid']
            else:
                print('Dataset not set')

            X2 = df_scaled.iloc[:,X2_vars]
            y2 = df_scaled.iloc[:,0]
            features_full1 = ['Age']
            for i in [X2_vars[1:]]:
                features_full1.extend(df_scaled.columns[i])
            features_demo = features_full1[0:3]

            #####################################################
            #####################################################
            #####################################################
            ### CRITICAL EDITS TO REMOVE BEHAVIORAL VARIABLES ###
            #####################################################
            #####################################################
            #####################################################

            if data == 3 or data == 4:
                if variable == 1:
                    X3 = df_scaled_i.iloc[:,X3_vars[10:]]
                elif variable == 2 or variable == 3:
                    X3 = df_scaled_i.iloc[:,X3_vars[12:]]
                else:
                    print('VARIABLE SELECTION ERROR')
                y3 = df_scaled_i.iloc[:,0]
                groups3 = df_scaled_i['fid'] # CHANGED FROM df_scaled.columns TO GET CORRECT # FAMID/GROUPS
                features_full2 = [] # REMOVED 'Age' from variable column names in features_full2
                if variable == 1:
                    for i in [X3_vars[10:]]:
                        features_full2.extend(df_scaled_i.columns[i])
                elif variable == 2 or variable == 3:
                    for i in [X3_vars[12:]]:
                        features_full2.extend(df_scaled_i.columns[i])
                else:
                    print('VARIABLE COLUMN NAME MATCHING ERROR')


            ##############################
            # CROSS-VALIDATION SELECTION #
            ##############################

            inner_rstate = 9052017
            outer_rstate = 12301985
            #inner_rstate = np.random.RandomState(9052017)
            #outer_rstate = np.random.RandomState(12301985)

            cv=GroupShuffleSplit
            #cv=GroupKFold
            cv_inner = cv(n_splits=20, test_size=0.2, random_state=inner_rstate)
            cv_outer = cv(n_splits=10, test_size=0.2, random_state=outer_rstate)
            n_iter=200
            n_iter_str = str(n_iter)

            print("Features Selected")
            print("Inner Training CV: ", cv_inner)
            print("Outer Testing CV: ", cv_outer)

            #print("Dataset: ", dataset_name)
            #print("Model Variable: ", model_var)
            #print("Model Estimator: ", model_estimator)
            #print("Imaging File: ", i_filename)

            ############################
            # SET SPECIFICITY OUTCOMES #
            ############################

            if dataset_name == "CLDRC_DropNA" and model_var == "READ":
                specificity_file = "PARTIAL CLDRC CROSS-CONSTRUCT READ SPECIFICITY RESULTS (N=1265)"
                df_outcomes = pd.read_csv("cldrc_1265_specificity_outcomes.csv")
            elif dataset_name == "CLDRC_DropNA" and (model_var == "IN" or model_var == "HI"):
                specificity_file = "PARTIAL CLDRC CROSS-CONSTRUCT IN/HI SPECIFICITY RESULTS (N=1190)"
                df_outcomes = pd.read_csv("cldrc_1190_specificity_outcomes.csv")
            elif dataset_name == "CLDRC_Imputed" and (model_var == "READ" or model_var == "IN" or model_var == "HI"):
                specificity_file = "FULL CLDRC CROSS-CONSTRUCT SPECIFICITY RESULTS (N=1675)"
                df_outcomes = pd.read_csv("cldrc_1675_specificity_outcomes.csv")
            elif dataset_name == "Hub_DropNA" and model_var == "READ":
                specificity_file = "PARTIAL HUB CROSS-CONSTRUCT READ SPECIFICITY RESULTS (N=214)"
                df_outcomes = pd.read_csv("hub_214_specificity_outcomes.csv")
            elif dataset_name == "Hub_DropNA" and (model_var == "IN" or model_var == "HI"):
                specificity_file = "PATIAL HUB CROSS-CONSTRUCT IN/HI SPECIFICITY RESULTS (N=209)"
                df_outcomes = pd.read_csv("hub_209_specificity_outcomes.csv")
            elif dataset_name == "Hub_Imputed" and (model_var == "READ" or model_var == "IN" or model_var == "HI"):
                specificity_file = "FULL HUB CROSS-CONSTRUCT SPECIFICITY RESULTS (N=324)"
                df_outcomes = pd.read_csv("hub_324_specificity_outcomes.csv")
            else:
                print("Dataset and Model Variables Not Set")

            print("")
            print("Specificity Filename: ", specificity_file)
            print("")
            r2 = df_outcomes['out_read_fs']
            i2 = df_outcomes['out_in_fs']
            h2 = df_outcomes['out_hi_fs']
            m2 = df_outcomes['out_math_fs']

            ###############################
            ### NESTED CROSS-VALIDATORS ###
            ###############################

            from sklearn.model_selection import ShuffleSplit

            cvreg=ShuffleSplit
            cvreg_inner = cvreg(n_splits=20, test_size=0.2, random_state=123085)
            cvreg_outer = cvreg(n_splits=9, test_size=0.2, random_state=90517)

            if (data == 3 or data == 4):
                #######################
                ### LassoCV Imaging ###
                #######################

###                print("LASSO_CV IMAGING")
###                search_time_start_lassoi = time.time()
###                alphas_lassocvi = np.logspace(-4, -0.3, 100)
###                lasso_cvi = LassoCV(alphas=alphas_lassocvi, cv=cvreg_inner, max_iter=10000, tol=0.0001, n_jobs=4)
###                lasso_cvi_score = []

###                for k, (trainCV, testCV) in enumerate(cvreg_outer.split(X3, y3, groups=groups3)):
###                    lasso_cvi.fit(X3.values[trainCV], y3[trainCV])
###                   print("[fold {0}] alpha: {1:.5f}, score: {2:.5f}".
###                          format(k, lasso_cvi.alpha_, lasso_cvi.score(X3.values[testCV], y3[testCV])))
###                    lasso_cvi_score.append(lasso_cvi.score(X3.values[testCV], y3[testCV]))
###                search_time_stop_lassoi = time.time()

###                feature_list_lassoi = []
###                for i in list(range(0,len(X3.columns))):
###                    feature_list_lassoi.append([lasso_cvi.coef_[i], X3.columns[i]])
###                    df_featimp_lassoi = pd.DataFrame(data=feature_list_lassoi, columns=['coef','feature'])
###                df_featimp_lassoi.reindex(df_featimp_lassoi.coef.abs().sort_values(ascending=False).index)
###                df_featimp_sum_lassoi = df_featimp_lassoi[(df_featimp_lassoi['coef'] > .001) | (df_featimp_lassoi['coef'] < -.001)]
###                print('')
#                print(df_featimp_sum_lassoi.reindex(df_featimp_sum_lassoi.coef.abs().sort_values(ascending=False).index))


                ###########################
                ### LassoLarsCV Imaging ###
                ###########################

                print("LASSOLARS_CV IMAGING")
                search_time_start_llarsi = time.time()
                llars_cvi = LassoLarsCV(cv=cvreg_inner, max_iter=10000, n_jobs=4)
                llars_cvi_score = []

                for k, (trainCV, testCV) in enumerate(cvreg_outer.split(X3, y3, groups=groups3)):
                    llars_cvi.fit(X3.values[trainCV], y3[trainCV])
                    print("[fold {0}] alpha: {1:.5f}, score: {2:.5f}".
                          format(k, llars_cvi.alpha_, llars_cvi.score(X3.values[testCV], y3[testCV])))
                    llars_cvi_score.append(llars_cvi.score(X3.values[testCV], y3[testCV]))
                search_time_stop_llarsi = time.time()

                feature_list_lli = []
                for i in list(range(0,len(X3.columns))):
                    feature_list_lli.append([llars_cvi.coef_[i], X3.columns[i]])
                    df_featimp_lli = pd.DataFrame(data=feature_list_lli, columns=['coef','feature'])
                df_featimp_lli.reindex(df_featimp_lli.coef.abs().sort_values(ascending=False).index)
                df_featimp_sum_lli = df_featimp_lli[(df_featimp_lli['coef'] > .001) | (df_featimp_lli['coef'] < -.001)]
                print('')
#                print(df_featimp_sum_lli.reindex(df_featimp_sum_lli.coef.abs().sort_values(ascending=False).index))
            else:
                print("Imaging regularization not available for CLDRC dataset")

            print('')
###            print("LassoCVI Mean: {0:.5f}; SD: {1:.5f}; alpha: {2:.5f}; time: {3:.3f}".
###                    format(np.mean(lasso_cvi_score), np.std(lasso_cvi_score), lasso_cvi.alpha_, search_time_stop_lassoi-search_time_start_lassoi))
            print("LLarsCVI Mean: {0:.5f}; SD: {1:.5f}; alpha: {2:.5f}; time: {3:.3f}".
                    format(np.mean(llars_cvi_score), np.std(llars_cvi_score), llars_cvi.alpha_, search_time_stop_llarsi-search_time_start_llarsi))
            print('')
###            print('Lasso Imaging Weights')
###            print(df_featimp_sum_lassoi.reindex(df_featimp_sum_lassoi.coef.abs().sort_values(ascending=False).index))
###            print('')
            print('LassoLars Imaging Weights')
            print(df_featimp_sum_lli.reindex(df_featimp_sum_lli.coef.abs().sort_values(ascending=False).index))
            print('')
            print('')
            print('')
            print('')
            print('')
            print('')

In [ ]:
with open("0.results_imagingonly_hubDKTfs5varloop_llars_READ_5.2.20.txt", 'w') as f:
    f.write(capture.stdout)

# LASSO-LLARS FS5 Dataset Variable Loop

In [ ]:
%%capture --no-stderr capture

for j in range(4,5): # MODIFY to (1,5) for all 4 datasets
    for z in range(1,4): # MODIFY to (1,4) for all 3 variables
        data = j
        variable = z
        estimator = 1
        imaging = 7
        #####################
        # DATASET SELECTION #
        #####################

        if data == 1 and variable == 1:
            dataset_name = "CLDRC_DropNA"
            model_var = "READ"
            df = pd.read_csv("df_vars_readfs_dropna_cldrc.csv", na_values = 'NaN', index_col=0)
        elif data == 1 and variable == 2:
            dataset_name = "CLDRC_DropNA"
            model_var = "IN"
            df = pd.read_csv("df_vars_infs_dropna_cldrc.csv", na_values = 'NaN', index_col=0)
        elif data == 1 and variable == 3:
            dataset_name = "CLDRC_DropNA"
            model_var = "HI"
            df = pd.read_csv("df_vars_hifs_dropna_cldrc.csv", na_values = 'NaN', index_col=0)
        elif data == 2 and variable == 1:
            dataset_name = "CLDRC_Imputed"
            model_var = "READ"
            df = pd.read_csv("df_vars_readfs_imputed_cldrc.csv", index_col=0)
        elif data == 2 and variable == 2:
            dataset_name = "CLDRC_Imputed"
            model_var = "IN"
            df = pd.read_csv("df_vars_infs_imputed_cldrc.csv", index_col=0)
        elif data == 2 and variable == 3:
            dataset_name = "CLDRC_Imputed"
            model_var = "HI"
            df = pd.read_csv("df_vars_hifs_imputed_cldrc.csv", index_col=0)
        elif data == 3 and variable == 1:
            dataset_name = "Hub_DropNA"
            model_var = "READ"
            df = pd.read_csv("df_vars_readfs_dropna_hub.csv", na_values = 'NaN', index_col=0)
        elif data == 3 and variable == 2:
            dataset_name = "Hub_DropNA"
            model_var = "IN"
            df = pd.read_csv("df_vars_infs_dropna_hub.csv", na_values = 'NaN', index_col=0)
        elif data == 3 and variable == 3:
            dataset_name = "Hub_DropNA"
            model_var = "HI"
            df = pd.read_csv("df_vars_hifs_dropna_hub.csv", na_values = 'NaN', index_col=0)
        elif data == 4 and variable == 1:
            dataset_name = "Hub_Imputed"
            model_var = "READ"
            df = pd.read_csv("df_vars_readfs_imputed_hub.csv", index_col=0)
        elif data == 4 and variable == 2:
            dataset_name = "Hub_Imputed"
            model_var = "IN"
            df = pd.read_csv("df_vars_infs_imputed_hub.csv", index_col=0)
        elif data == 4 and variable == 3:
            dataset_name = "Hub_Imputed"
            model_var = "HI"
            df = pd.read_csv("df_vars_hifs_imputed_hub.csv", index_col=0)
        else:
            print("Data and Variable Not Selected")

        #######################
        # IMPORT IMAGING DATA #
        #######################

        if (data == 3 or data == 4):
            if imaging == 1:
                df_i = pd.read_csv('1.fs5new.csv', na_values='NaN', index_col=0)
                i_filename = "fs5_labels"
            elif imaging == 2:
                df_i = pd.read_csv('1.tbssall.csv', na_values='NaN', index_col=0)                      # CAN REMOVE ALL FROM tbss
                i_filename = "tbss"
            elif imaging == 3:
                df_i = pd.read_csv('1.aseg.csv', na_values='NaN', index_col=0)
                i_filename = "aseg"
            elif imaging == 4:
                df_i = pd.read_csv('1.fs5new_tbssall.csv', na_values='NaN', index_col=0)                     # CAN REMOVE ALL FROM tbss
        #        df_fs5 = pd.read_csv('fs5_labels_NaN_reduced.csv', na_values='NaN', index_col=0)
        #        df_tbss = pd.read_csv('tbss_metrics_NaN_FA.csv', na_values='NaN', index_col=0)
        #        df_i = df_fs5.merge(df_tbss, on='scanid1')
                i_filename = "fs5_tbss"
            elif imaging == 5:
                df_i = pd.read_csv('1.fs5new_aseg.csv', na_values='NaN', index_col=0)
        #        df_fs5 = pd.read_csv('fs5_labels_NaN_reduced.csv', na_values='NaN', index_col=0)
        #        df_aseg = pd.read_csv('DKTaseg_hub.csv', na_values='NaN', index_col=0)
                df_i = df_fs5.merge(df_aseg, on='scanid1')
                i_filename = "fs5_aseg"
            elif imaging == 6:
                df_i = pd.read_csv('1.tbssall_aseg.csv', na_values='NaN', index_col=0)                    # CAN REMOVE ALL FROM tbss
        #        df_tbss = pd.read_csv('tbss_metrics_NaN_FA.csv', na_values='NaN', index_col=0)
        #        df_aseg = pd.read_csv('DKTaseg_hub.csv', na_values='NaN', index_col=0)
        #        df_i = df_tbss.merge(df_aseg, on='scanid1')
                i_filename = "tbss_aseg"
            elif imaging == 7:
                df_i = pd.read_csv('1.fs5new_tbssall_aseg.csv', na_values='NaN', index_col=0)                # CAN REMOVE ALL FROM tbss
        #        df_fs5 = pd.read_csv('fs5_labels_NaN_reduced.csv', na_values='NaN', index_col=0)
        #        df_tbss = pd.read_csv('tbss_metrics_NaN_FA.csv', na_values='NaN', index_col=0)
        #        df_aseg = pd.read_csv('DKTaseg_hub.csv', na_values='NaN', index_col=0)
        #        df_pre_i = df_fs5.merge(df_tbss, on='scanid1')
        #        df_i = df_pre_i.merge(df_aseg, on='scanid1')
                i_filename = "fs5_tbss_aseg"
            else:
                print('No imaging selected for Hub analysis')
        else:
            df_i = pd.read_csv('1.fs5new_tbssall_aseg.csv', na_values='NaN', index_col=0)
            df_with_i = pd.read_csv('1.fs5new_tbssall_aseg.csv', na_values='NaN', index_col=0)
            print('No imaging in CLDRC')

        ################################
        # COMBINE BEHAVIORAL & IMAGING #
        ################################

        if data == 3 or data == 4:
            df_with_i = df.merge(df_i, on='scanid1')
            df_with_i = df_with_i.dropna()
        #    df_with_i.drop('Unnamed: 0_y', axis=1, inplace=True)
        #    df.rename(columns={'Unnamed: 0_x':'Unnamed: 0'}, inplace=True)
        #    df_with_i.drop(df_with_i.columns[13], axis=1, inplace=True)
        #    df_with_i.drop('Unnamed: 0_y', axis=1, inplace=True)
        #    df_with_i.reset_index(drop=True,inplace=True)

        else:
            i_filename = 'No_Imaging_in_Model'
        print("Imaging File: ", i_filename)
        print("Dataset Name: ", dataset_name)
        print("Model Variable: ", model_var)

        if 'Unnamed: 0' in df.columns:
            print('Unnamed: 0 in df')
        if 'Unnamed: 0' in df_i.columns:
            print('Unnamed: 0 in df_i')
        if 'Unnamed: 0' in df_with_i.columns:
            print('Unnamed: 0 in df_with_i')

        ######################
        # SETTING ORIG_INDEX #
        ######################

        df.reset_index(inplace=True)
        df_i.reset_index(inplace=True)
        df_with_i.reset_index(inplace=True)

        df.rename(columns={'index':'orig_index'}, inplace=True)
        df_i.rename(columns={'index':'orig_index'}, inplace=True)
        df_with_i.rename(columns={'index':'orig_index'}, inplace=True)

        #######################
        # ESTIMATOR SELECTION #
        #######################

        if estimator == 1:
            model_estimator = "Linear_SVR"
            param_dist = {"C": uniform(.0001,5),
                          "epsilon": uniform(0,1)}
            alg = LinearSVR(max_iter=10000)
        elif estimator == 2:
            model_estimator = "SVR_Linear_Kernel"
            param_dist = {"C": uniform(.0001,5),
                          "epsilon": uniform(0,1)}
            alg = SVR(kernel = 'linear')
        elif estimator == 3:
            model_estimator = "SVR_Nonlinear_RBF"
            param_dist = {"C": uniform(.0001,5),
                         "epsilon": uniform(0,1)}
            alg = SVR(kernel = 'rbf', gamma = 'auto')
        elif estimator == 4:
            model_estimator="XGBoost"
            param_dist = {"n_estimators": randint(5,100),
                          "learning_rate": uniform(0.01,0.5),
                          "gamma": uniform(0.0,0.5),
                          "max_depth": randint(3,10),
                          "min_child_weight": randint(1,8),
                          "subsample": uniform(.6,0.4),
                          "colsample_bytree": uniform(.3,0.7),
                          "reg_lambda": uniform(0.001,5),
                          "reg_alpha": uniform(0.00001,1)}
            alg = XGBRegressor(n_jobs=6)
        elif estimator == 5:
            model_estimator = "Linear_Regression"
            alg = linear_model.LinearRegression(n_jobs=8)
        else:
            print("Estimator Not Selected")

        print("Model Estimator: ", model_estimator)

        #########################
        # DATASET PREPROCESSING #
        #########################

        baselength = len(df.columns)
        base = [3,4]
        base2 = list(range(6,len(df.columns)))
        base.extend(base2)
        full1_preprocessor = make_column_transformer(
            (StandardScaler(), base),
            remainder='passthrough')

        df.rename(columns={'Unnamed: 0':'orig_index'}, inplace=True)
        ###################################
        ### POSSIBLY SHOULD BE EXCLUDED ###
        ###################################
        if data == 3 or data == 4:
            df_with_i.rename(columns={'Unnamed: 0':'orig_index'}, inplace=True)
        ###################################
        ### POSSIBLY SHOULD BE EXCLUDED ###
        ###################################

        i_index1 = index1 = df.columns.values[3]
        index2 = df.columns.values[6:].tolist()
        i_index3 = index3 = df.columns.values[5]
        i_index4 = index4 = df.columns.values[0:3].tolist()
        i_index5 = index5 = df.columns.values[4]

        index2.extend(index4)
        index2.insert(0,index1)
        index2.insert(1,index5)
        index2.extend([index3])

        df_scaled = full1_preprocessor.fit_transform(df)
        df_scaled = pd.DataFrame(data=df_scaled)
        df_scaled.columns = index2

        if data == 3 or data == 4:
            base3 = list(range(6,len(df_i.columns)+baselength-2))

            full = [3,4]
            full.extend(base3)
            full2_preprocessor = make_column_transformer(
                (StandardScaler(), full),
                remainder='passthrough')

            i_index2 = df_with_i.columns.values[6:].tolist()
            i_index2.extend(i_index4)
            i_index2.insert(0,i_index1)
            i_index2.insert(1,i_index5)
            i_index2.extend([i_index3])

            df_scaled_i = full2_preprocessor.fit_transform(df_with_i)
            df_scaled_i = pd.DataFrame(data=df_scaled_i)
            df_scaled_i.columns = i_index2

        print("Data Preprocessed - Scaler Applied")

        ######################
        # FEATURES SELECTION #
        ######################

        x1 = [1,2]
        x2 = [len(df_scaled.columns)-1]
        x3 = list(range(3,len(df_scaled.columns)-4))
        X1_vars = x1 + x2
        X2_vars = x1 + x2 + x3

        if data == 3 or data ==4:
            x5 = [len(df_scaled_i.columns)-1]
            x4 = list(range(len(df_scaled.columns)-4,len(df_scaled_i.columns)-4))
            X3_vars = x1 + x5 + x3 + x4

        X1 = df_scaled.iloc[:,X1_vars]
        y1 = df_scaled.iloc[:,0]
        if data == 1 or data == 2:
            groups1 = groups2 = df_scaled['famid']
        elif data == 3 or data == 4:
            groups1 = groups2 = df_scaled['fid']
        else:
            print('Dataset not set')

        X2 = df_scaled.iloc[:,X2_vars]
        y2 = df_scaled.iloc[:,0]
        features_full1 = ['Age']
        for i in [X2_vars[1:]]:
            features_full1.extend(df_scaled.columns[i])
        features_demo = features_full1[0:3]

        if data == 3 or data == 4:
            X3 = df_scaled_i.iloc[:,X3_vars]
            y3 = df_scaled_i.iloc[:,0]
            groups3 = df_scaled_i['fid'] # CHANGED FROM df_scaled.columns TO GET CORRECT # FAMID/GROUPS
            features_full2 = ['Age']
            for i in [X3_vars[1:]]:
                features_full2.extend(df_scaled_i.columns[i])


        ##############################
        # CROSS-VALIDATION SELECTION #
        ##############################

        inner_rstate = 9052017
        outer_rstate = 12301985
        #inner_rstate = np.random.RandomState(9052017)
        #outer_rstate = np.random.RandomState(12301985)

        cv=GroupShuffleSplit
        #cv=GroupKFold
        cv_inner = cv(n_splits=20, test_size=0.2, random_state=inner_rstate)
        cv_outer = cv(n_splits=10, test_size=0.2, random_state=outer_rstate)
        n_iter=200
        n_iter_str = str(n_iter)

        print("Features Selected")
        print("Inner Training CV: ", cv_inner)
        print("Outer Testing CV: ", cv_outer)

        print("Dataset: ", dataset_name)
        print("Model Variable: ", model_var)
        print("Model Estimator: ", model_estimator)
        print("Imaging File: ", i_filename)

        ############################
        # SET SPECIFICITY OUTCOMES #
        ############################

        if dataset_name == "CLDRC_DropNA" and model_var == "READ":
            specificity_file = "PARTIAL CLDRC CROSS-CONSTRUCT READ SPECIFICITY RESULTS (N=1265)"
            df_outcomes = pd.read_csv("cldrc_1265_specificity_outcomes.csv")
        elif dataset_name == "CLDRC_DropNA" and (model_var == "IN" or model_var == "HI"):
            specificity_file = "PARTIAL CLDRC CROSS-CONSTRUCT IN/HI SPECIFICITY RESULTS (N=1190)"
            df_outcomes = pd.read_csv("cldrc_1190_specificity_outcomes.csv")
        elif dataset_name == "CLDRC_Imputed" and (model_var == "READ" or model_var == "IN" or model_var == "HI"):
            specificity_file = "FULL CLDRC CROSS-CONSTRUCT SPECIFICITY RESULTS (N=1675)"
            df_outcomes = pd.read_csv("cldrc_1675_specificity_outcomes.csv")
        elif dataset_name == "Hub_DropNA" and model_var == "READ":
            specificity_file = "PARTIAL HUB CROSS-CONSTRUCT READ SPECIFICITY RESULTS (N=214)"
            df_outcomes = pd.read_csv("hub_214_specificity_outcomes.csv")
        elif dataset_name == "Hub_DropNA" and (model_var == "IN" or model_var == "HI"):
            specificity_file = "PATIAL HUB CROSS-CONSTRUCT IN/HI SPECIFICITY RESULTS (N=209)"
            df_outcomes = pd.read_csv("hub_209_specificity_outcomes.csv")
        elif dataset_name == "Hub_Imputed" and (model_var == "READ" or model_var == "IN" or model_var == "HI"):
            specificity_file = "FULL HUB CROSS-CONSTRUCT SPECIFICITY RESULTS (N=324)"
            df_outcomes = pd.read_csv("hub_324_specificity_outcomes.csv")
        else:
            print("Dataset and Model Variables Not Set")

        print("")
        print("Specificity Filename: ", specificity_file)
        print("")
        r2 = df_outcomes['out_read_fs']
        i2 = df_outcomes['out_in_fs']
        h2 = df_outcomes['out_hi_fs']
        m2 = df_outcomes['out_math_fs']

        ###############################
        ### NESTED CROSS-VALIDATORS ###
        ###############################

        from sklearn.model_selection import ShuffleSplit

        cvreg=ShuffleSplit
        cvreg_inner = cvreg(n_splits=20, test_size=0.2, random_state=123085)
        cvreg_outer = cvreg(n_splits=9, test_size=0.2, random_state=90517)

        ##########################
        ### LassoCV Behavioral ###
        ##########################

###        from sklearn.linear_model import LassoCV

###        print("LASSO_CV BEHAVIOR")
###        search_time_start_lassob = time.time()
###        alphas_lassocvb = np.logspace(-4, -0.3, 100)
###        lasso_cvb = LassoCV(alphas=alphas_lassocvb, cv=cvreg_inner, max_iter=100000, tol=0.0001, n_jobs=4)
###        lasso_cvb_score = []

###        for k, (trainCV, testCV) in enumerate(cvreg_outer.split(X2, y2, groups=groups2)):
###            lasso_cvb.fit(X2.values[trainCV], y2[trainCV])
###            print("[fold {0}] alpha: {1:.5f}, score: {2:.5f}".
###                  format(k, lasso_cvb.alpha_, lasso_cvb.score(X2.values[testCV], y2[testCV])))
###            lasso_cvb_score.append(lasso_cvb.score(X2.values[testCV], y2[testCV]))
###        search_time_stop_lassob = time.time()

###        feature_list_lassob = []
###        for i in list(range(0,len(X2.columns))):
###            feature_list_lassob.append([lasso_cvb.coef_[i], X2.columns[i]])
###            df_featimp_lassob = pd.DataFrame(data=feature_list_lassob, columns=['coef','feature'])
###        df_featimp_lassob.reindex(df_featimp_lassob.coef.abs().sort_values(ascending=False).index)
###        df_featimp_sum_lassob = df_featimp_lassob[(df_featimp_lassob['coef'] > .001) | (df_featimp_lassob['coef'] < -.001)]
###        print(df_featimp_sum_lassob.reindex(df_featimp_sum_lassob.coef.abs().sort_values(ascending=False).index))
###        print("")
###        print("")


        ##############################
        ### LassoLarsCV Behavioral ###
        ##############################

        from sklearn.linear_model import LassoLarsCV

        print("LASSOLARS_CV BEHAVIOR")
        search_time_start_llarsb = time.time()
        llars_cvb = LassoLarsCV(cv=cvreg_inner, max_iter=100000, n_jobs=4)
        llars_cvb_score = []

        for k, (trainCV, testCV) in enumerate(cvreg_outer.split(X1, y1, groups=groups1)):
            llars_cvb.fit(X1.values[trainCV], y1[trainCV])
            print("[fold {0}] alpha: {1:.5f}, score: {2:.5f}".
                  format(k, llars_cvb.alpha_, llars_cvb.score(X1.values[testCV], y1[testCV])))
            llars_cvb_score.append(llars_cvb.score(X1.values[testCV], y1[testCV]))
        search_time_stop_llarsb = time.time()


        feature_list_llb = []
        for i in list(range(0,len(X1.columns))):
            feature_list_llb.append([llars_cvb.coef_[i], X1.columns[i]])
            df_featimp_llb = pd.DataFrame(data=feature_list_llb, columns=['coef','feature'])
        df_featimp_llb.reindex(df_featimp_llb.coef.abs().sort_values(ascending=False).index)
        df_featimp_sum_llb = df_featimp_llb[(df_featimp_llb['coef'] > .001) | (df_featimp_llb['coef'] < -.001)]
        print(df_featimp_sum_llb.reindex(df_featimp_sum_llb.coef.abs().sort_values(ascending=False).index))
        print("")
        print("")


###        if (data == 3 or data == 4):
            #######################
            ### LassoCV Imaging ###
            #######################

###            print("LASSO_CV IMAGING")
###            search_time_start_lassoi = time.time()
###            alphas_lassocvi = np.logspace(-4, -0.3, 100)
###            lasso_cvi = LassoCV(alphas=alphas_lassocvi, cv=cvreg_inner, max_iter=100000, tol=0.0001, n_jobs=4)
###            lasso_cvi_score = []

###            for k, (trainCV, testCV) in enumerate(cvreg_outer.split(X3, y3, groups=groups3)):
###                lasso_cvi.fit(X3.values[trainCV], y3[trainCV])
###                print("[fold {0}] alpha: {1:.5f}, score: {2:.5f}".
###                      format(k, lasso_cvi.alpha_, lasso_cvi.score(X3.values[testCV], y3[testCV])))
###                lasso_cvi_score.append(lasso_cvi.score(X3.values[testCV], y3[testCV]))
###            search_time_stop_lassoi = time.time()

###            feature_list_lassoi = []
###            for i in list(range(0,len(X3.columns))):
###                feature_list_lassoi.append([lasso_cvi.coef_[i], X3.columns[i]])
###                df_featimp_lassoi = pd.DataFrame(data=feature_list_lassoi, columns=['coef','feature'])
###            df_featimp_lassoi.reindex(df_featimp_lassoi.coef.abs().sort_values(ascending=False).index)
###            df_featimp_sum_lassoi = df_featimp_lassoi[(df_featimp_lassoi['coef'] > .001) | (df_featimp_lassoi['coef'] < -.001)]
###            print(df_featimp_sum_lassoi.reindex(df_featimp_sum_lassoi.coef.abs().sort_values(ascending=False).index))


            ###########################
            ### LassoLarsCV Imaging ###
            ###########################

###            print("LASSOLARS_CV IMAGING")
###            search_time_start_llarsi = time.time()
###            llars_cvi = LassoLarsCV(cv=cvreg_inner, max_iter=10000, n_jobs=4)
###            llars_cvi_score = []

###            for k, (trainCV, testCV) in enumerate(cvreg_outer.split(X3, y3, groups=groups3)):
###                llars_cvi.fit(X3.values[trainCV], y3[trainCV])
###                print("[fold {0}] alpha: {1:.5f}, score: {2:.5f}".
###                      format(k, llars_cvi.alpha_, llars_cvi.score(X3.values[testCV], y3[testCV])))
###                llars_cvi_score.append(llars_cvi.score(X3.values[testCV], y3[testCV]))
###            search_time_stop_llarsi = time.time()

###            feature_list_lli = []
###            for i in list(range(0,len(X3.columns))):
###                feature_list_lli.append([llars_cvi.coef_[i], X3.columns[i]])
###                df_featimp_lli = pd.DataFrame(data=feature_list_lli, columns=['coef','feature'])
###            df_featimp_lli.reindex(df_featimp_lli.coef.abs().sort_values(ascending=False).index)
###            df_featimp_sum_lli = df_featimp_lli[(df_featimp_lli['coef'] > .001) | (df_featimp_lli['coef'] < -.001)]
###            print(df_featimp_sum_lli.reindex(df_featimp_sum_lli.coef.abs().sort_values(ascending=False).index))
###        else:
###            print("Imaging regularization not available for CLDRC dataset")


###        print("LassoCVB Mean: {0:.5f}; SD: {1:.5f}; alpha: {2:.5f}; time: {3:.3f}".
###              format(np.mean(lasso_cvb_score), np.std(lasso_cvb_score), lasso_cvb.alpha_, search_time_stop_lassob-search_time_start_lassob))
###        if data == 3 or data ==4:
###            print("LassoCVI Mean: {0:.5f}; SD: {1:.5f}; alpha: {2:.5f}; time: {3:.3f}".
###                  format(np.mean(lasso_cvi_score), np.std(lasso_cvi_score), lasso_cvi.alpha_, search_time_stop_lassoi-search_time_start_lassoi))
        print("LLarsCVB Mean: {0:.5f}; SD: {1:.5f}; alpha: {2:.5f}; time: {3:.3f}".
              format(np.mean(llars_cvb_score), np.std(llars_cvb_score), llars_cvb.alpha_, search_time_stop_llarsb-search_time_start_llarsb))
###        if data == 3 or data ==4:
###            print("LLarsCVI Mean: {0:.5f}; SD: {1:.5f}; alpha: {2:.5f}; time: {3:.3f}".
###                  format(np.mean(llars_cvi_score), np.std(llars_cvi_score), llars_cvi.alpha_, search_time_stop_llarsi-search_time_start_llarsi))

###        print('Lasso Behavior Weights')
###        print(df_featimp_sum_lassob.reindex(df_featimp_sum_lassob.coef.abs().sort_values(ascending=False).index))
###        print('')
###        if data == 3 or data ==4:
###            print('Lasso Imaging Weights')
###            print(df_featimp_sum_lassoi.reindex(df_featimp_sum_lassoi.coef.abs().sort_values(ascending=False).index))
###            print('')
        print('LassoLars Behavior Weights')
        print(df_featimp_sum_llb.reindex(df_featimp_sum_llb.coef.abs().sort_values(ascending=False).index))
        print('')
###        if data == 3 or data ==4:
###            print('LassoLars Imaging Weights')
###            print(df_featimp_sum_lli.reindex(df_featimp_sum_lli.coef.abs().sort_values(ascending=False).index))
###            print('')

In [ ]:
with open("0.results_demoonly_varloop_llars.txt", 'w') as f:
    f.write(capture.stdout)

# ESTIMATOR/SPECIFICITY, Dataset, Variable Loop

In [ ]:
%%capture --no-stderr capture

for j in range(1,5): # CURRENTLY JUST HUB IMPUTED!!! MODIFY to (1,5) for all 4 datasets
    for z in range(1,2): # READ
#    for z in range(2,3): # IN
#    for z in range(3,4): # HI
        for q in range(2,6):
            data = j
            variable = z
            estimator = q
            imaging = 7
            #####################
            # DATASET SELECTION #
            #####################

            if data == 1 and variable == 1:
                dataset_name = "CLDRC_DropNA"
                model_var = "READ"
                df = pd.read_csv("df_vars_readfs_dropna_cldrc.csv", na_values = 'NaN', index_col=0)
            elif data == 1 and variable == 2:
                dataset_name = "CLDRC_DropNA"
                model_var = "IN"
                df = pd.read_csv("df_vars_infs_dropna_cldrc.csv", na_values = 'NaN', index_col=0)
            elif data == 1 and variable == 3:
                dataset_name = "CLDRC_DropNA"
                model_var = "HI"
                df = pd.read_csv("df_vars_hifs_dropna_cldrc.csv", na_values = 'NaN', index_col=0)
            elif data == 2 and variable == 1:
                dataset_name = "CLDRC_Imputed"
                model_var = "READ"
                df = pd.read_csv("df_vars_readfs_imputed_cldrc.csv", index_col=0)
            elif data == 2 and variable == 2:
                dataset_name = "CLDRC_Imputed"
                model_var = "IN"
                df = pd.read_csv("df_vars_infs_imputed_cldrc.csv", index_col=0)
            elif data == 2 and variable == 3:
                dataset_name = "CLDRC_Imputed"
                model_var = "HI"
                df = pd.read_csv("df_vars_hifs_imputed_cldrc.csv", index_col=0)
            elif data == 3 and variable == 1:
                dataset_name = "Hub_DropNA"
                model_var = "READ"
                df = pd.read_csv("df_vars_readfs_dropna_hub.csv", na_values = 'NaN', index_col=0)
            elif data == 3 and variable == 2:
                dataset_name = "Hub_DropNA"
                model_var = "IN"
                df = pd.read_csv("df_vars_infs_dropna_hub.csv", na_values = 'NaN', index_col=0)
            elif data == 3 and variable == 3:
                dataset_name = "Hub_DropNA"
                model_var = "HI"
                df = pd.read_csv("df_vars_hifs_dropna_hub.csv", na_values = 'NaN', index_col=0)
            elif data == 4 and variable == 1:
                dataset_name = "Hub_Imputed"
                model_var = "READ"
                df = pd.read_csv("df_vars_readfs_imputed_hub.csv", index_col=0)
            elif data == 4 and variable == 2:
                dataset_name = "Hub_Imputed"
                model_var = "IN"
                df = pd.read_csv("df_vars_infs_imputed_hub.csv", index_col=0)
            elif data == 4 and variable == 3:
                dataset_name = "Hub_Imputed"
                model_var = "HI"
                df = pd.read_csv("df_vars_hifs_imputed_hub.csv", index_col=0)
            else:
                print("Data and Variable Not Selected")

            #######################
            # IMPORT IMAGING DATA #
            #######################

            if (data == 3 or data == 4):
                if imaging == 1:
                    df_i = pd.read_csv('1.fs5new.csv', na_values='NaN', index_col=0)
                    i_filename = "fs5_labels"
                elif imaging == 2:
                    df_i = pd.read_csv('1.tbssall.csv', na_values='NaN', index_col=0)                      # CAN REMOVE ALL FROM tbss
                    i_filename = "tbss"
                elif imaging == 3:
                    df_i = pd.read_csv('1.aseg.csv', na_values='NaN', index_col=0)
                    i_filename = "aseg"
                elif imaging == 4:
                    df_i = pd.read_csv('1.fs5new_tbssall.csv', na_values='NaN', index_col=0)                     # CAN REMOVE ALL FROM tbss
            #        df_fs5 = pd.read_csv('fs5_labels_NaN_reduced.csv', na_values='NaN', index_col=0)
            #        df_tbss = pd.read_csv('tbss_metrics_NaN_FA.csv', na_values='NaN', index_col=0)
            #        df_i = df_fs5.merge(df_tbss, on='scanid1')
                    i_filename = "fs5_tbss"
                elif imaging == 5:
                    df_i = pd.read_csv('1.fs5new_aseg.csv', na_values='NaN', index_col=0)
            #        df_fs5 = pd.read_csv('fs5_labels_NaN_reduced.csv', na_values='NaN', index_col=0)
            #        df_aseg = pd.read_csv('DKTaseg_hub.csv', na_values='NaN', index_col=0)
                    df_i = df_fs5.merge(df_aseg, on='scanid1')
                    i_filename = "fs5_aseg"
                elif imaging == 6:
                    df_i = pd.read_csv('1.tbssall_aseg.csv', na_values='NaN', index_col=0)                    # CAN REMOVE ALL FROM tbss
            #        df_tbss = pd.read_csv('tbss_metrics_NaN_FA.csv', na_values='NaN', index_col=0)
            #        df_aseg = pd.read_csv('DKTaseg_hub.csv', na_values='NaN', index_col=0)
            #        df_i = df_tbss.merge(df_aseg, on='scanid1')
                    i_filename = "tbss_aseg"
                elif imaging == 7:
                    df_i = pd.read_csv('1.fs5new_tbssall_aseg.csv', na_values='NaN', index_col=0)                # CAN REMOVE ALL FROM tbss
            #        df_fs5 = pd.read_csv('fs5_labels_NaN_reduced.csv', na_values='NaN', index_col=0)
            #        df_tbss = pd.read_csv('tbss_metrics_NaN_FA.csv', na_values='NaN', index_col=0)
            #        df_aseg = pd.read_csv('DKTaseg_hub.csv', na_values='NaN', index_col=0)
            #        df_pre_i = df_fs5.merge(df_tbss, on='scanid1')
            #        df_i = df_pre_i.merge(df_aseg, on='scanid1')
                    i_filename = "fs5_tbss_aseg"
                else:
                    print('No imaging selected for Hub analysis')
            else:
                df_i = pd.read_csv('1.fs5new_tbssall_aseg.csv', na_values='NaN', index_col=0)
                df_with_i = pd.read_csv('1.fs5new_tbssall_aseg.csv', na_values='NaN', index_col=0)
                print('No imaging in CLDRC')

            ################################
            # COMBINE BEHAVIORAL & IMAGING #
            ################################

            if data == 3 or data == 4:
                df_with_i = df.merge(df_i, on='scanid1')
                df_with_i = df_with_i.dropna()
            #    df_with_i.drop('Unnamed: 0_y', axis=1, inplace=True)
            #    df.rename(columns={'Unnamed: 0_x':'Unnamed: 0'}, inplace=True)
            #    df_with_i.drop(df_with_i.columns[13], axis=1, inplace=True)
            #    df_with_i.drop('Unnamed: 0_y', axis=1, inplace=True)
            #    df_with_i.reset_index(drop=True,inplace=True)

            else:
                i_filename = 'No_Imaging_in_Model'
#            print("Imaging File: ", i_filename)
#            print("Dataset Name: ", dataset_name)
#            print("Model Variable: ", model_var)

            if 'Unnamed: 0' in df.columns:
                print('Unnamed: 0 in df')
            if 'Unnamed: 0' in df_i.columns:
                print('Unnamed: 0 in df_i')
            if 'Unnamed: 0' in df_with_i.columns:
                print('Unnamed: 0 in df_with_i')

            ######################
            # SETTING ORIG_INDEX #
            ######################

            df.reset_index(inplace=True)
            df_i.reset_index(inplace=True)
            df_with_i.reset_index(inplace=True)

            df.rename(columns={'index':'orig_index'}, inplace=True)
            df_i.rename(columns={'index':'orig_index'}, inplace=True)
            df_with_i.rename(columns={'index':'orig_index'}, inplace=True)

            #######################
            # ESTIMATOR SELECTION #
            #######################

            if estimator == 1:
                model_estimator = "Linear_SVR"
                param_dist = {"C": uniform(.0001,5),
                              "epsilon": uniform(0,1)}
                alg = LinearSVR(max_iter=10000)
            elif estimator == 2:
                model_estimator = "SVR_Linear_Kernel"
                param_dist = {"C": uniform(.0001,5),
                              "epsilon": uniform(0,1)}
                alg = SVR(kernel = 'linear')
            elif estimator == 3:
                model_estimator = "SVR_Nonlinear_RBF"
                param_dist = {"C": uniform(.0001,5),
                             "epsilon": uniform(0,1)}
                alg = SVR(kernel = 'rbf', gamma = 'auto')
            elif estimator == 4:
                model_estimator="XGBoost"
                param_dist = {"n_estimators": randint(5,100),
                              "learning_rate": uniform(0.01,0.5),
                              "gamma": uniform(0.0,0.5),
                              "max_depth": randint(3,10),
                              "min_child_weight": randint(1,8),
                              "subsample": uniform(.6,0.4),
                              "colsample_bytree": uniform(.3,0.7),
                              "reg_lambda": uniform(0.001,5),
                              "reg_alpha": uniform(0.00001,1)}
                alg = XGBRegressor(n_jobs=6)
            elif estimator == 5:
                model_estimator = "Linear_Regression"
                alg = linear_model.LinearRegression(n_jobs=8)
            else:
                print("Estimator Not Selected")

            print("Model Estimator: ", model_estimator)

            #########################
            # DATASET PREPROCESSING #
            #########################

            baselength = len(df.columns)
            base = [3,4]
            base2 = list(range(6,len(df.columns)))
            base.extend(base2)
            full1_preprocessor = make_column_transformer(
                (StandardScaler(), base),
                remainder='passthrough')

            df.rename(columns={'Unnamed: 0':'orig_index'}, inplace=True)
            ###################################
            ### POSSIBLY SHOULD BE EXCLUDED ###
            ###################################
            if data == 3 or data == 4:
                df_with_i.rename(columns={'Unnamed: 0':'orig_index'}, inplace=True)
            ###################################
            ### POSSIBLY SHOULD BE EXCLUDED ###
            ###################################

            i_index1 = index1 = df.columns.values[3]
            index2 = df.columns.values[6:].tolist()
            i_index3 = index3 = df.columns.values[5]
            i_index4 = index4 = df.columns.values[0:3].tolist()
            i_index5 = index5 = df.columns.values[4]

            index2.extend(index4)
            index2.insert(0,index1)
            index2.insert(1,index5)
            index2.extend([index3])

            df_scaled = full1_preprocessor.fit_transform(df)
            df_scaled = pd.DataFrame(data=df_scaled)
            df_scaled.columns = index2

            if data == 3 or data == 4:
                base3 = list(range(6,len(df_i.columns)+baselength-2))

                full = [3,4]
                full.extend(base3)
                full2_preprocessor = make_column_transformer(
                    (StandardScaler(), full),
                    remainder='passthrough')

                i_index2 = df_with_i.columns.values[6:].tolist()
                i_index2.extend(i_index4)
                i_index2.insert(0,i_index1)
                i_index2.insert(1,i_index5)
                i_index2.extend([i_index3])

                df_scaled_i = full2_preprocessor.fit_transform(df_with_i)
                df_scaled_i = pd.DataFrame(data=df_scaled_i)
                df_scaled_i.columns = i_index2

            print("Data Preprocessed - Scaler Applied")

            ######################
            # FEATURES SELECTION #
            ######################

            x1 = [1,2]
            x2 = [len(df_scaled.columns)-1]
            x3 = list(range(3,len(df_scaled.columns)-4))
            X1_vars = x1 + x2
            X2_vars = x1 + x2 + x3

            if data == 3 or data ==4:
                x5 = [len(df_scaled_i.columns)-1]
                x4 = list(range(len(df_scaled.columns)-4,len(df_scaled_i.columns)-4))
                X3_vars = x1 + x5 + x3 + x4

            X1 = df_scaled.iloc[:,X1_vars]
            y1 = df_scaled.iloc[:,0]
            if data == 1 or data == 2:
                groups1 = groups2 = df_scaled['famid']
            elif data == 3 or data == 4:
                groups1 = groups2 = df_scaled['fid']
            else:
                print('Dataset not set')

            X2 = df_scaled.iloc[:,X2_vars]
            y2 = df_scaled.iloc[:,0]
            features_full1 = ['Age']
            for i in [X2_vars[1:]]:
                features_full1.extend(df_scaled.columns[i])
            features_demo = features_full1[0:3]

            if data == 3 or data == 4:
                X3 = df_scaled_i.iloc[:,X3_vars]
                y3 = df_scaled_i.iloc[:,0]
                groups3 = df_scaled_i['fid'] # CHANGED FROM df_scaled.columns TO GET CORRECT # FAMID/GROUPS
                features_full2 = ['Age']
                for i in [X3_vars[1:]]:
                    features_full2.extend(df_scaled_i.columns[i])


            ##############################
            # CROSS-VALIDATION SELECTION #
            ##############################

            inner_rstate = 9052017
            outer_rstate = 12301985
            #inner_rstate = np.random.RandomState(9052017)
            #outer_rstate = np.random.RandomState(12301985)

            cv=GroupShuffleSplit
            #cv=GroupKFold
            cv_inner = cv(n_splits=20, test_size=0.2, random_state=inner_rstate)
            cv_outer = cv(n_splits=10, test_size=0.2, random_state=outer_rstate)
            n_iter=200
            n_iter_str = str(n_iter)

            print("Features Selected")
#            print("Inner Training CV: ", cv_inner)
#            print("Outer Testing CV: ", cv_outer)

#            print("Dataset: ", dataset_name)
#            print("Model Variable: ", model_var)
#            print("Model Estimator: ", model_estimator)
#            print("Imaging File: ", i_filename)

            ############################
            # SET SPECIFICITY OUTCOMES #
            ############################

            if dataset_name == "CLDRC_DropNA" and model_var == "READ":
                specificity_file = "PARTIAL CLDRC CROSS-CONSTRUCT READ SPECIFICITY RESULTS (N=1265)"
                df_outcomes = pd.read_csv("cldrc_1265_specificity_outcomes.csv")
            elif dataset_name == "CLDRC_DropNA" and (model_var == "IN" or model_var == "HI"):
                specificity_file = "PARTIAL CLDRC CROSS-CONSTRUCT IN/HI SPECIFICITY RESULTS (N=1190)"
                df_outcomes = pd.read_csv("cldrc_1190_specificity_outcomes.csv")
            elif dataset_name == "CLDRC_Imputed" and (model_var == "READ" or model_var == "IN" or model_var == "HI"):
                specificity_file = "FULL CLDRC CROSS-CONSTRUCT SPECIFICITY RESULTS (N=1675)"
                df_outcomes = pd.read_csv("cldrc_1675_specificity_outcomes.csv")
            elif dataset_name == "Hub_DropNA" and model_var == "READ":
                specificity_file = "PARTIAL HUB CROSS-CONSTRUCT READ SPECIFICITY RESULTS (N=214)"
                df_outcomes = pd.read_csv("hub_214_specificity_outcomes.csv")
            elif dataset_name == "Hub_DropNA" and (model_var == "IN" or model_var == "HI"):
                specificity_file = "PATIAL HUB CROSS-CONSTRUCT IN/HI SPECIFICITY RESULTS (N=209)"
                df_outcomes = pd.read_csv("hub_209_specificity_outcomes.csv")
            elif dataset_name == "Hub_Imputed" and (model_var == "READ" or model_var == "IN" or model_var == "HI"):
                specificity_file = "FULL HUB CROSS-CONSTRUCT SPECIFICITY RESULTS (N=324)"
                df_outcomes = pd.read_csv("hub_324_specificity_outcomes.csv")
            else:
                print("Dataset and Model Variables Not Set")

#            print("")
            print("Specificity Filename: ", specificity_file)
            print("")
            r2 = df_outcomes['out_read_fs']
            i2 = df_outcomes['out_in_fs']
            h2 = df_outcomes['out_hi_fs']
            m2 = df_outcomes['out_math_fs']

            #########################################
            ### RESULTS WITH DEMOGRAPHIC FEATURES ###
            #########################################

#            print("Dataset: ", dataset_name)
#            print("Model Variable: ", model_var)
#            print("Model Estimator: ", model_estimator)
#            print("")

#            print("Model Tuning Times: \n")

            for train1, test1 in cv_inner.split(X1, y1, groups=groups1):
                X_train1, X_test1 = X1.values[train1], X1.values[test1]
                y_train1, y_test1 = y1[train1], y1[test1]
                groups_train1, groups_test1 = groups1[train1], groups1[test1]

            search_time_start1 = time.time()
#            print("Demographics tuning time...")
            if estimator == 5:
                clf1 = alg
            else:
                clf1 = RandomizedSearchCV(alg, param_dist, n_iter=n_iter, iid=False, cv=cv_inner, n_jobs=4)
                clf1.fit(X_train1, y_train1, groups = groups_train1)

            search_time_stop1 = time.time()
#            print('%.3f' %(search_time_stop1 - search_time_start1))
#            print("")

            if estimator == 5:
                cvalscorer1 = clf1
            else:
                cvalscorer1 = clf1.best_estimator_

            train1_scores = cross_val_score(cvalscorer1, X_train1, y_train1, groups=groups_train1, cv=cv_inner, n_jobs=4)
            test1_scores = cross_val_score(cvalscorer1, X_test1, y_test1, groups=groups_test1, cv=cv_outer, n_jobs=4)

            #################################
            ### RESULTS WITH ALL FEATURES ###
            #################################

            for train2, test2 in cv_inner.split(X2, y2, groups=groups2):
                X_train2, X_test2 = X2.values[train2], X2.values[test2]
                y_train2, y_test2 = y2[train2], y2[test2]
                groups_train2, groups_test2 = groups2[train2], groups2[test2]
                r_train2, r_test2 = r2[train2], r2[test2]
                i_train2, i_test2 = i2[train2], i2[test2]
                h_train2, h_test2 = h2[train2], h2[test2]
                m_train2, m_test2 = m2[train2], m2[test2]

            search_time_start2 = time.time()
#            print("Full Model tuning time...")
            if estimator == 5:
                clf2 = alg
            else:
                clf2 = RandomizedSearchCV(alg, param_dist, n_iter=n_iter, iid=False, cv=cv_inner, n_jobs=4)
                clf2.fit(X_train2, y_train2, groups = groups_train2)

            search_time_stop2 = time.time()
#            print('%.3f' %(search_time_stop2 - search_time_start2))
#            print("\n"*2)

            if estimator == 5:
                cvalscorer2 = clf2
            else:
                cvalscorer2 = clf2.best_estimator_

            train2_scores = cross_val_score(cvalscorer2, X_train2, y_train2, groups=groups_train2, cv=cv_inner, n_jobs=4)
            test2_scores = cross_val_score(cvalscorer2, X_test2, y_test2, groups=groups_test2, cv=cv_outer, n_jobs=4)

#            print("DEMOGRAPHIC MODEL RESULTS")
#            print("")
#            print("Train [inner CV] R2s:\n", train1_scores)
#            print("Train R2s Mean:\n%.3f" %train1_scores.mean())
#            print("Train R2s SD:\n%.3f" %train1_scores.std())
#            print("")
#            print("Test [outer CV] R2s:\n", test1_scores)
#            print("Test R2s Mean:\n%.3f" %test1_scores.mean())
#            print("Test R2s SD:\n%.3f" %test1_scores.std())
#            print("")
            if estimator != 5:
                print("Best Demo Estimator:\n", clf1.best_estimator_)
                print("")
                print("Best Demo Model params:")
                best_params1 = clf1.best_params_
                for param_name in sorted(best_params1.keys()):
                    print('%s: %r' % (param_name, best_params1[param_name]))
#            print("\n"*2)
#            print("FULL BEHAVIORAL MODEL RESULTS")
#            print("")
#            print("Train [inner CV] R2s:\n", train2_scores)
#            print("Train R2s Mean:\n%.3f" %train2_scores.mean())
#            print("Train R2s SD:\n%.3f" %train2_scores.std())
#            print("")
#            print("Test [outer CV] R2s:\n", test2_scores)
#            print("Test R2s Mean:\n%.3f" %test2_scores.mean())
#            print("Test R2s SD:\n%.3f" %test2_scores.std())
#            print("")
            if estimator != 5:
                print("Best Full Estimator:\n", clf2.best_estimator_)
                print("")
                print("Best Full params:")
                best_params2 = clf2.best_params_
                for param_name in sorted(best_params2.keys()):
                    print('%s: %r' % (param_name, best_params2[param_name]))

            ###########################################
            ### RESULTS WITH ALL FEATURES & IMAGING ###
            ###########################################

            if data == 3 or data == 4:
#                print("Imaging File: ", i_filename)
#                print("")

                for train3, test3 in cv_inner.split(X3, y3, groups=groups3):
                    X_train3, X_test3 = X3.values[train3], X3.values[test3]
                    y_train3, y_test3 = y3[train3], y3[test3]
                    groups_train3, groups_test3 = groups3[train3], groups3[test3]
                    r_train3, r_test3 = r2[train3], r2[test3]
                    i_train3, i_test3 = i2[train3], i2[test3]
                    h_train3, h_test3 = h2[train3], h2[test3]
                    m_train3, m_test3 = m2[train3], m2[test3]

#                print("Full Model & Imaging tuning time...")
                search_time_start3 = time.time()
                if estimator == 5:
                    clf3 = alg
                else:
                    clf3 = RandomizedSearchCV(alg, param_dist, n_iter=n_iter, iid=False, cv=cv_inner, n_jobs=4)
                    clf3.fit(X_train3, y_train3, groups = groups_train3)
                search_time_stop3 = time.time()
#                print('%.3f' %(search_time_stop3 - search_time_start3))
#                print("\n"*3)

                if estimator == 5:
                    cvalscorer3 = clf3
                else:
                    cvalscorer3 = clf3.best_estimator_

                train3_scores = cross_val_score(cvalscorer3, X_train3, y_train3, groups=groups_train3, cv=cv_inner, n_jobs=4)
                test3_scores = cross_val_score(cvalscorer3, X_test3, y_test3, groups=groups_test3, cv=cv_outer, n_jobs=4)

#                print("FULL BEHAVIORAL & IMAGING MODEL RESULTS")
#                print("")
#                print("Train [inner CV] R2s:\n", train3_scores)
#                print("Train R2s Mean:\n%.3f" %train3_scores.mean())
#                print("Train R2s SD:\n%.3f" %train3_scores.std())
#                print("")
#                print("Test [outer CV] R2s:\n", test3_scores)
#                print("Test R2s Mean:\n%.3f" %test3_scores.mean())
#                print("Test R2s SD:\n%.3f" %test3_scores.std())
#                print("")
                if estimator != 5:
                    print("Best Full Estimator:\n", clf3.best_estimator_)
                    print("")
                    print("Best Full params:")
                    best_params3 = clf3.best_params_
                    for param_name in sorted(best_params3.keys()):
                        print('%s: %r' % (param_name, best_params3[param_name]))
            else:
                print('No Imaging Data in CLDRC')

            #########################################
            ####  DISCRIMINANT VALIDITY RESULTS  ####
            #########################################

#            print("SPECIFICITY RESULTS - TEST SET")
#            print("")
#            print("Dataset: ", dataset_name)
#            print("Model Variable: ", model_var)
#            print("Model Estimator: ", model_estimator)

#            print("\n"*2)

            if estimator == 5:
                fullsample_scores_test = cross_val_score(clf2, X_test2, y_test2, groups=groups_test2, cv=cv_outer, n_jobs=8)
                var_read_scores_test = cross_val_score(clf2, X_test2, r_test2, groups=groups_test2, cv=cv_outer, n_jobs=8)
                var_in_scores_test = cross_val_score(clf2, X_test2, i_test2, groups=groups_test2, cv=cv_outer, n_jobs=8)
                var_hi_scores_test = cross_val_score(clf2, X_test2, h_test2, groups=groups_test2, cv=cv_outer, n_jobs=8)
                var_math_scores_test = cross_val_score(clf2, X_test2, m_test2, groups=groups_test2, cv=cv_outer, n_jobs=8)
            else:
                fullsample_scores_test = cross_val_score(clf2.best_estimator_, X_test2, y_test2, groups=groups_test2, cv=cv_outer, n_jobs=8)
                var_read_scores_test = cross_val_score(clf2.best_estimator_, X_test2, r_test2, groups=groups_test2, cv=cv_outer, n_jobs=8)
                var_in_scores_test = cross_val_score(clf2.best_estimator_, X_test2, i_test2, groups=groups_test2, cv=cv_outer, n_jobs=8)
                var_hi_scores_test = cross_val_score(clf2.best_estimator_, X_test2, h_test2, groups=groups_test2, cv=cv_outer, n_jobs=8)
                var_math_scores_test = cross_val_score(clf2.best_estimator_, X_test2, m_test2, groups=groups_test2, cv=cv_outer, n_jobs=8)

#            print("SPECIFICITY CORRELATIONS:")
#            print("")
#            print(model_var + "-READ Test R Mean:\n%.3f" %math.sqrt(abs(var_read_scores_test.mean())))
#            print(model_var + "-IN Test R Mean:\n%.3f" %math.sqrt(abs(var_in_scores_test.mean())))
#            print(model_var + "-HI Test R Mean:\n%.3f" %math.sqrt(abs(var_hi_scores_test.mean())))
#            print(model_var + "-MATH Test R Mean:\n%.3f" %math.sqrt(abs(var_math_scores_test.mean())))
#            print("\n"*2)

#            print("VARIANCES EXPLAINED:")
#            print("")
#            print(model_var + "-" + model_var + " Test R2s Mean:\n%.3f" %fullsample_scores_test.mean())
#            print(model_var + "-" + model_var + " Test R2s SD:\n%.3f" %fullsample_scores_test.std())
#            print("")
            #print(model_var + "-READ R2s:\n", var_read_scores)
#            print(model_var + "-READ Test R2s Mean:\n%.3f" %var_read_scores_test.mean())
#            print(model_var + "-READ Test R2s SD:\n%.3f" %var_read_scores_test.std())
#            print("")
            #print(model_var + "-IN R2s:\n", var_in_scores)
#            print(model_var + "-IN Test R2s Mean:\n%.3f" %var_in_scores_test.mean())
#            print(model_var + "-IN Test R2s SD:\n%.3f" %var_in_scores_test.std())
#            print("")
            #print(model_var + "-HI R2s:\n", var_hi_scores)
#            print(model_var + "-HI Test R2s Mean:\n%.3f" %var_hi_scores_test.mean())
#            print(model_var + "-HI Test R2s SD:\n%.3f" %var_hi_scores_test.std())
#            print("")
            #print(model_var + "-MATH R2s:\n", var_math_scores)
#            print(model_var + "-MATH Test R2s Mean:\n%.3f" %var_math_scores_test.mean())
#            print(model_var + "-MATH Test R2s SD:\n%.3f" %var_math_scores_test.std())

            # Results of Training sample specificity:

#            print("SPECIFICITY RESULTS - TRAIN SET")
#            print("Dataset: ", dataset_name)
#            print("")

            if estimator == 5:
                fullsample_scores_train = cross_val_score(clf2, X_train2, y_train2, groups=groups_train2, cv=cv_outer, n_jobs=8)
                var_read_scores_train = cross_val_score(clf2, X_train2, r_train2, groups=groups_train2, cv=cv_outer, n_jobs=8)
                var_in_scores_train = cross_val_score(clf2, X_train2, i_train2, groups=groups_train2, cv=cv_outer, n_jobs=8)
                var_hi_scores_train = cross_val_score(clf2, X_train2, h_train2, groups=groups_train2, cv=cv_outer, n_jobs=8)
                var_math_scores_train = cross_val_score(clf2, X_train2, m_train2, groups=groups_train2, cv=cv_outer, n_jobs=8)
            else:
                fullsample_scores_train = cross_val_score(clf2.best_estimator_, X_train2, y_train2, groups=groups_train2, cv=cv_outer, n_jobs=8)
                var_read_scores_train = cross_val_score(clf2.best_estimator_, X_train2, r_train2, groups=groups_train2, cv=cv_outer, n_jobs=8)
                var_in_scores_train = cross_val_score(clf2.best_estimator_, X_train2, i_train2, groups=groups_train2, cv=cv_outer, n_jobs=8)
                var_hi_scores_train = cross_val_score(clf2.best_estimator_, X_train2, h_train2, groups=groups_train2, cv=cv_outer, n_jobs=8)
                var_math_scores_train = cross_val_score(clf2.best_estimator_, X_train2, m_train2, groups=groups_train2, cv=cv_outer, n_jobs=8)

#            print(model_var + "-" + model_var + " Train R Mean:\n%.3f" %math.sqrt(abs(fullsample_scores_train.mean())))
#            print(model_var + "-READ Train R Mean:\n%.3f" %math.sqrt(abs(var_read_scores_train.mean())))
#            print(model_var + "-IN Train R Mean:\n%.3f" %math.sqrt(abs(var_in_scores_train.mean())))
#            print(model_var + "-HI Train R Mean:\n%.3f" %math.sqrt(abs(var_hi_scores_train.mean())))
#            print(model_var + "-MATH Train R Mean:\n%.3f" %math.sqrt(abs(var_math_scores_train.mean())))

            # Results of FULL sample specificity:
            if data == 2 or data == 4:
                if estimator == 5:
                    fullsample_scores_full = cross_val_score(clf2, X2, y2, groups=groups2, cv=cv_outer, n_jobs=8)
                    var_read_scores_full = cross_val_score(clf2, X2, r2, groups=groups2, cv=cv_outer, n_jobs=8)
                    var_in_scores_full = cross_val_score(clf2, X2, i2, groups=groups2, cv=cv_outer, n_jobs=8)
                    var_hi_scores_full = cross_val_score(clf2, X2, h2, groups=groups2, cv=cv_outer, n_jobs=8)
                    var_math_scores_full = cross_val_score(clf2, X2, m2, groups=groups2, cv=cv_outer, n_jobs=8)
                else:
                    fullsample_scores_full = cross_val_score(clf2.best_estimator_, X2, y2, groups=groups2, cv=cv_outer, n_jobs=8)
                    var_read_scores_full = cross_val_score(clf2.best_estimator_, X2, r2, groups=groups2, cv=cv_outer, n_jobs=8)
                    var_in_scores_full = cross_val_score(clf2.best_estimator_, X2, i2, groups=groups2, cv=cv_outer, n_jobs=8)
                    var_hi_scores_full = cross_val_score(clf2.best_estimator_, X2, h2, groups=groups2, cv=cv_outer, n_jobs=8)
                    var_math_scores_full = cross_val_score(clf2.best_estimator_, X2, m2, groups=groups2, cv=cv_outer, n_jobs=8)
#                print("SPECIFICITY RESULTS - TEST + TRAIN SETS")
#                print("Dataset: ", dataset_name)
#                print("")

#                print(model_var + "-" + model_var + " Full Sample R Mean:\n%.3f" %math.sqrt(abs(fullsample_scores_full.mean())))
#                print(model_var + "-READ Full Sample R Mean:\n%.3f" %math.sqrt(abs(var_read_scores_full.mean())))
#                print(model_var + "-IN Full Sample R Mean:\n%.3f" %math.sqrt(abs(var_in_scores_full.mean())))
#                print(model_var + "-HI Full Sample R Mean:\n%.3f" %math.sqrt(abs(var_hi_scores_full.mean())))
#                print(model_var + "-MATH Full Sample R Mean:\n%.3f" %math.sqrt(abs(var_math_scores_full.mean())))
            else:
                print("Dataset is not imputed for estimation of full-sample correlations")

            #################################
            ###   FULL CONTENT TO PRINT   ###
            #################################

            print("Dataset_Name: ", dataset_name)
            print("Model_Variable: ", model_var)
            print("Model_Estimator: ", model_estimator)
            print("Imaging_File: ", i_filename)
            print("")
            print("Data_Preprocessed_-_Scaler_Applied")
            print("Features_Selected")
            print("Inner_Training_CV: ", cv_inner)
            print("Outer_Testing_CV: ", cv_outer)
            print("Specificity_Filename: ", specificity_file)
            print("")

            print("Model_Tuning_Times: \n")
            print("Demographics_tuning_time: ")
            print("%.3f" %(search_time_stop1 - search_time_start1))
            print("")
            print("Full_Model_tuning_time: ")
            print("%.3f" %(search_time_stop2 - search_time_start2))
            print("")

            if data == 3 or data == 4:
                print("Full_Model_&_Imaging_tuning_time: ")
                print("%.3f" %(search_time_stop3 - search_time_start3))
            print("")

            print("DEMOGRAPHIC_MODEL")
            print("")
            if estimator == 5:
                print("No Best Estimator for LinearRegression")
            else:
                print("Demo_Best_Estimator:\n", clf1.best_estimator_)
            print("")
            print("Demo_Train_R2s_Mean:\n%.3f" %train1_scores.mean())
            print("Demo_Train_R2s_SD:\n%.3f" %train1_scores.std())
            print("")
            print("Demo_Test_R2s_Mean:\n%.3f" %test1_scores.mean())
            print("Demo_Test_R2s_SD:\n%.3f" %test1_scores.std())
            print("")

            print("FULL_MODEL")
            print("")
            if estimator == 5:
                print("No Best Estimator for LinearRegression")
            else:
                print("Full_Best_Estimator:\n", clf2.best_estimator_)
            print("")
            print("Full_Train_R2s_Mean:\n%.3f" %train2_scores.mean())
            print("Full_Train_R2s_SD:\n%.3f" %train2_scores.std())
            print("")
            print("Full_Test_R2s_Mean:\n%.3f" %test2_scores.mean())
            print("Full_Test_R2s_SD:\n%.3f" %test2_scores.std())
            print("")

            if data == 3 or data == 4:
                print("FULL_&_IMAGING_MODEL")
                print("")
                if estimator == 5:
                    print("No Best Estimator for LinearRegression")
                else:
                    print("Imaging_Best_Estimator:\n", clf3.best_estimator_)
                print("")
                print("Imaging_Train_R2s_Mean:\n%.3f" %train3_scores.mean())
                print("Imaging_Train_R2s_SD:\n%.3f" %train3_scores.std())
                print("")
                print("Imaging_Test_R2s_Mean:\n%.3f" %test3_scores.mean())
                print("Imaging_Test_R2s_SD:\n%.3f" %test3_scores.std())
                print("")

            print("SPECIFICITY_CORRELATIONS_-_TEST_SET")
            print("")
            print(model_var + "-READ_Test_R_Mean:\n%.3f" %math.sqrt(abs(var_read_scores_test.mean())))
            print(model_var + "-IN_Test_R_Mean:\n%.3f" %math.sqrt(abs(var_in_scores_test.mean())))
            print(model_var + "-HI_Test_R_Mean:\n%.3f" %math.sqrt(abs(var_hi_scores_test.mean())))
            print(model_var + "-MATH_Test_R_Mean:\n%.3f" %math.sqrt(abs(var_math_scores_test.mean())))
            print("")
            print("SPECIFICITY_CORRELATIONS_-_TRAIN_SET")
            print("")
            print(model_var + "-READ_Train_R_Mean:\n%.3f" %math.sqrt(abs(var_read_scores_train.mean())))
            print(model_var + "-IN_Train_R_Mean:\n%.3f" %math.sqrt(abs(var_in_scores_train.mean())))
            print(model_var + "-HI_Train_R_Mean:\n%.3f" %math.sqrt(abs(var_hi_scores_train.mean())))
            print(model_var + "-MATH_Train_R_Mean:\n%.3f" %math.sqrt(abs(var_math_scores_train.mean())))
            print("")

            if data == 2 or data == 4:
                print("SPECIFICITY_CORRELATIONS_-_TEST_+_TRAIN_SETS")
                print(model_var + "-READ_Full_Sample_R_Mean:\n%.3f" %math.sqrt(abs(var_read_scores_full.mean())))
                print(model_var + "-IN_Full_Sample_R_Mean:\n%.3f" %math.sqrt(abs(var_in_scores_full.mean())))
                print(model_var + "-HI_Full_Sample_R_Mean:\n%.3f" %math.sqrt(abs(var_hi_scores_full.mean())))
                print(model_var + "-MATH_Full_Sample_R_Mean:\n%.3f" %math.sqrt(abs(var_math_scores_full.mean())))
                print("")


In [ ]:
with open("0.results_hubvarloop_estimators_specificity_READ.txt", 'w') as f:
    f.write(capture.stdout)

#with open("0.results_hubvarloop_estimators_specificity_IN.txt", 'w') as f:
#    f.write(capture.stdout)

#with open("0.results_hubvarloop_estimators_specificity_HI.txt", 'w') as f:
#    f.write(capture.stdout)

# DIAGNOSTIC ACCURACY Estimator (Data/Var) Loop

In [ ]:
%%capture --no-stderr capture

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.svm import SVC, LinearSVC
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, classification_report,  f1_score

#for j in range(1,5): # MODIFY to (1,5) for all 4 datasets
for j in [1,2,4]: # ALL DATA EXCEPT HUBDROPNA
#    for z in range(1,2): # READ
#    for z in range(2,3): # IN
    for z in range(3,4): # HI
#    for z in range(4,5): # ADHD
#    for z in range(5,6): # ADHD TYPE
        for q in range(2,6):
            data = j
            variable = z
            estimator = q
            imaging = 7

            #####################
            # DATASET SELECTION #
            #####################

            if data == 1 and variable == 1:
                dataset_name = "CLDRC_DropNA_dx"
                model_var = "READ"
                df = pd.read_csv("df_vars_readfs_dropna_cldrc_dx.csv", na_values = 'NaN', index_col=0)
            elif data == 1 and variable == 2:
                dataset_name = "CLDRC_DropNA_dx"
                model_var = "IN"
                df = pd.read_csv("df_vars_infs_dropna_cldrc_dx.csv", na_values = 'NaN', index_col=0)
            elif data == 1 and variable == 3:
                dataset_name = "CLDRC_DropNA_dx"
                model_var = "HI"
                df = pd.read_csv("df_vars_hifs_dropna_cldrc_dx.csv", na_values = 'NaN', index_col=0)

            elif data == 1 and variable == 4:
                dataset_name = "CLDRC_DropNA_dx"
                model_var = "ADHD"
                df = pd.read_csv("df_vars_adhdfs_dropna_cldrc_dx.csv", na_values = 'NaN', index_col=0)
            elif data == 1 and variable == 5:
                dataset_name = "CLDRC_DropNA_dx"
                model_var = "ADHD TYPE"
                df = pd.read_csv("df_vars_adhdtype_dropna_cldrc_dx.csv", na_values = 'NaN', index_col=0)


            elif data == 2 and variable == 1:
                dataset_name = "CLDRC_Imputed_dx"
                model_var = "READ"
                df = pd.read_csv("df_vars_readfs_imputed_cldrc_dx.csv", index_col=0)
            elif data == 2 and variable == 2:
                dataset_name = "CLDRC_Imputed_dx"
                model_var = "IN"
                df = pd.read_csv("df_vars_infs_imputed_cldrc_dx.csv", index_col=0)
            elif data == 2 and variable == 3:
                dataset_name = "CLDRC_Imputed_dx"
                model_var = "HI"
                df = pd.read_csv("df_vars_hifs_imputed_cldrc_dx.csv", index_col=0)

            elif data == 2 and variable == 4:
                dataset_name = "CLDRC_Imputed_dx"
                model_var = "ADHD"
                df = pd.read_csv("df_vars_adhdfs_imputed_cldrc_dx.csv", index_col=0)
            elif data == 2 and variable == 5:
                dataset_name = "CLDRC_Imputed_dx"
                model_var = "ADHD TYPE"
                df = pd.read_csv("df_vars_adhdtype_imputed_cldrc_dx.csv", index_col=0)


            elif data == 3 and variable == 1:
                dataset_name = "Hub_DropNA_dx"
                model_var = "READ"
                df = pd.read_csv("df_vars_readfs_dropna_hub_dx.csv", na_values = 'NaN', index_col=0)
            elif data == 3 and variable == 2:
                dataset_name = "Hub_DropNA_dx"
                model_var = "IN"
                df = pd.read_csv("df_vars_infs_dropna_hub_dx.csv", na_values = 'NaN', index_col=0)
            elif data == 3 and variable == 3:
                dataset_name = "Hub_DropNA_dx"
                model_var = "HI"
                df = pd.read_csv("df_vars_hifs_dropna_hub_dx.csv", na_values = 'NaN', index_col=0)

            elif data == 3 and variable == 4:
                dataset_name = "Hub_DropNA_dx"
                model_var = "ADHD"
                df = pd.read_csv("df_vars_adhdfs_dropna_hub_dx.csv", na_values = 'NaN', index_col=0)
            elif data == 3 and variable == 5:
                dataset_name = "Hub_DropNA_dx"
                model_var = "ADHD TYPE"
                df = pd.read_csv("df_vars_adhdtype_dropna_hub_dx.csv", na_values = 'NaN', index_col=0)


            elif data == 4 and variable == 1:
                dataset_name = "Hub_Imputed_dx"
                model_var = "READ"
                df = pd.read_csv("df_vars_readfs_imputed_hub_dx.csv", index_col=0)
            elif data == 4 and variable == 2:
                dataset_name = "Hub_Imputed_dx"
                model_var = "IN"
                df = pd.read_csv("df_vars_infs_imputed_hub_dx.csv", index_col=0)
            elif data == 4 and variable == 3:
                dataset_name = "Hub_Imputed_dx"
                model_var = "HI"
                df = pd.read_csv("df_vars_hifs_imputed_hub_dx.csv", index_col=0)

            elif data == 4 and variable == 4:
                dataset_name = "Hub_Imputed_dx"
                model_var = "ADHD"
                df = pd.read_csv("df_vars_adhdfs_imputed_hub_dx.csv", index_col=0)
            elif data == 4 and variable == 5:
                dataset_name = "Hub_Imputed_dx"
                model_var = "ADHD TYPE"
                df = pd.read_csv("df_vars_adhdtype_imputed_hub_dx.csv", index_col=0)


            else:
                print("Data and Variable Not Selected")

            #######################
            # IMPORT IMAGING DATA #
            #######################

            if (data == 3 or data == 4):
                if imaging == 1:
                    df_i = pd.read_csv('1.fs5new.csv', na_values='NaN', index_col=0)
                    i_filename = "fs5_labels"
                elif imaging == 2:
                    df_i = pd.read_csv('1.tbssall.csv', na_values='NaN', index_col=0)                      # CAN REMOVE ALL FROM tbss
                    i_filename = "tbss"
                elif imaging == 3:
                    df_i = pd.read_csv('1.aseg.csv', na_values='NaN', index_col=0)
                    i_filename = "aseg"
                elif imaging == 4:
                    df_i = pd.read_csv('1.fs5new_tbssall.csv', na_values='NaN', index_col=0)                     # CAN REMOVE ALL FROM tbss
            #        df_fs5 = pd.read_csv('fs5_labels_NaN_reduced.csv', na_values='NaN', index_col=0)
            #        df_tbss = pd.read_csv('tbss_metrics_NaN_FA.csv', na_values='NaN', index_col=0)
            #        df_i = df_fs5.merge(df_tbss, on='scanid1')
                    i_filename = "fs5_tbss"
                elif imaging == 5:
                    df_i = pd.read_csv('1.fs5new_aseg.csv', na_values='NaN', index_col=0)
            #        df_fs5 = pd.read_csv('fs5_labels_NaN_reduced.csv', na_values='NaN', index_col=0)
            #        df_aseg = pd.read_csv('DKTaseg_hub.csv', na_values='NaN', index_col=0)
                    df_i = df_fs5.merge(df_aseg, on='scanid1')
                    i_filename = "fs5_aseg"
                elif imaging == 6:
                    df_i = pd.read_csv('1.tbssall_aseg.csv', na_values='NaN', index_col=0)                    # CAN REMOVE ALL FROM tbss
            #        df_tbss = pd.read_csv('tbss_metrics_NaN_FA.csv', na_values='NaN', index_col=0)
            #        df_aseg = pd.read_csv('DKTaseg_hub.csv', na_values='NaN', index_col=0)
            #        df_i = df_tbss.merge(df_aseg, on='scanid1')
                    i_filename = "tbss_aseg"
                elif imaging == 7:
                    df_i = pd.read_csv('1.fs5new_tbssall_aseg.csv', na_values='NaN', index_col=0)                # CAN REMOVE ALL FROM tbss
            #        df_fs5 = pd.read_csv('fs5_labels_NaN_reduced.csv', na_values='NaN', index_col=0)
            #        df_tbss = pd.read_csv('tbss_metrics_NaN_FA.csv', na_values='NaN', index_col=0)
            #        df_aseg = pd.read_csv('DKTaseg_hub.csv', na_values='NaN', index_col=0)
            #        df_pre_i = df_fs5.merge(df_tbss, on='scanid1')
            #        df_i = df_pre_i.merge(df_aseg, on='scanid1')
                    i_filename = "fs5_tbss_aseg"
                else:
                    print('No imaging selected for Hub analysis')
            else:
                df_i = pd.read_csv('1.fs5new_tbssall_aseg.csv', na_values='NaN', index_col=0)
                df_with_i = pd.read_csv('1.fs5new_tbssall_aseg.csv', na_values='NaN', index_col=0)
                print('No imaging in CLDRC')

            ################################
            # COMBINE BEHAVIORAL & IMAGING #
            ################################

            if data == 3 or data == 4:
                df_with_i = df.merge(df_i, on='scanid1')
                df_with_i = df_with_i.dropna()
            #    df_with_i.drop('Unnamed: 0_y', axis=1, inplace=True)
            #    df.rename(columns={'Unnamed: 0_x':'Unnamed: 0'}, inplace=True)
            #    df_with_i.drop(df_with_i.columns[13], axis=1, inplace=True)
            #    df_with_i.drop('Unnamed: 0_y', axis=1, inplace=True)
            #    df_with_i.reset_index(drop=True,inplace=True)

            else:
                i_filename = 'No_Imaging_in_Model'
#            print("Imaging File: ", i_filename)
#            print("Dataset Name: ", dataset_name)
#            print("Model Variable: ", model_var)

            if 'Unnamed: 0' in df.columns:
                print('Unnamed: 0 in df')
            if 'Unnamed: 0' in df_i.columns:
                print('Unnamed: 0 in df_i')
            if 'Unnamed: 0' in df_with_i.columns:
                print('Unnamed: 0 in df_with_i')

            ######################
            # SETTING ORIG_INDEX #
            ######################

            df.reset_index(inplace=True)
            df_i.reset_index(inplace=True)
            df_with_i.reset_index(inplace=True)

            df.rename(columns={'index':'orig_index'}, inplace=True)
            df_i.rename(columns={'index':'orig_index'}, inplace=True)
            df_with_i.rename(columns={'index':'orig_index'}, inplace=True)

            #######################
            # ESTIMATOR SELECTION #
            #######################

            if estimator == 1:
                model_estimator = "Linear_SVC"
                param_dist = {"C": uniform(.0001,5)}
                alg = LinearSVC(penalty='l2', max_iter=10000, loss='squared_hinge') ### CHANGE l2 for l1 OR elasticnet ### CHANGE hinge for squared_hinge
            elif estimator == 2:
                model_estimator = "SVC_Linear_Kernel"
                param_dist = {"C": uniform(.0001,5)}
                alg = SVC(kernel = 'linear', class_weight='balanced')
            elif estimator == 3:
                model_estimator = "SVC_Nonlinear_RBF"
                param_dist = {"C": uniform(.0001,5)}
                alg = SVC(kernel = 'rbf', class_weight='balanced', gamma = 'auto')
            elif estimator == 4:
                model_estimator="XGBoost"
                param_dist = {"n_estimators": randint(5,100),
                              "learning_rate": uniform(0.01,0.5),
                              "gamma": uniform(0.0,0.5),
                              "max_depth": randint(3,10),
                              "min_child_weight": randint(1,8),
                              "subsample": uniform(.6,0.4),
                              "colsample_bytree": uniform(.3,0.7),
                              "reg_lambda": uniform(0.001,5),
                              "reg_alpha": uniform(0.00001,1)}
                alg = XGBClassifier(n_jobs=6)
            elif estimator == 5:
                model_estimator = "Logistic_Regression"
                alg = linear_model.LogisticRegression(penalty='l1', max_iter=10000, solver='liblinear', class_weight='balanced') ### CHANGE l2 for l1 OR elasticnet (set l1_ratio) ###
                param_dist = {"C": uniform(.0001,5)}#,
                              #"l1_ratio": uniform(0,1)}
                #MAY NEED multi_class='multinomial' and solver='lbfgs'

            else:
                print("Estimator Not Selected")

            print("Model Estimator: ", model_estimator)

            #########################
            # DATASET PREPROCESSING #
            #########################

            baselength = len(df.columns)
            base = [3,4]
            base2 = list(range(6,len(df.columns)))
            base.extend(base2)
            full1_preprocessor = make_column_transformer(
                (StandardScaler(), base),
                remainder='passthrough')

            df.rename(columns={'Unnamed: 0':'orig_index'}, inplace=True)
            ###################################
            ### POSSIBLY SHOULD BE EXCLUDED ###
            ###################################
            if data == 3 or data == 4:
                df_with_i.rename(columns={'Unnamed: 0':'orig_index'}, inplace=True)
            ###################################
            ### POSSIBLY SHOULD BE EXCLUDED ###
            ###################################

            i_index1 = index1 = df.columns.values[3]
            index2 = df.columns.values[6:].tolist()
            i_index3 = index3 = df.columns.values[5]
            i_index4 = index4 = df.columns.values[0:3].tolist()
            i_index5 = index5 = df.columns.values[4]

            index2.extend(index4)
            index2.insert(0,index1)
            index2.insert(1,index5)
            index2.extend([index3])

            df_scaled = full1_preprocessor.fit_transform(df)
            df_scaled = pd.DataFrame(data=df_scaled)
            df_scaled.columns = index2

            if data == 3 or data == 4:
                base3 = list(range(6,len(df_i.columns)+baselength-2))

                full = [3,4]
                full.extend(base3)
                full2_preprocessor = make_column_transformer(
                    (StandardScaler(), full),
                    remainder='passthrough')

                i_index2 = df_with_i.columns.values[6:].tolist()
                i_index2.extend(i_index4)
                i_index2.insert(0,i_index1)
                i_index2.insert(1,i_index5)
                i_index2.extend([i_index3])

                df_scaled_i = full2_preprocessor.fit_transform(df_with_i)
                df_scaled_i = pd.DataFrame(data=df_scaled_i)
                df_scaled_i.columns = i_index2

            print("Data Preprocessed - Scaler Applied")

            ######################
            # FEATURES SELECTION #
            ######################

            x1 = [1,2]
            x2 = [len(df_scaled.columns)-1]
            x3 = list(range(3,len(df_scaled.columns)-4))
            X1_vars = x1 + x2
            X2_vars = x1 + x2 + x3

            if data == 3 or data ==4:
                x5 = [len(df_scaled_i.columns)-1]
                x4 = list(range(len(df_scaled.columns)-4,len(df_scaled_i.columns)-4))
                X3_vars = x1 + x5 + x3 + x4

            X1 = df_scaled.iloc[:,X1_vars]
            y1 = df.iloc[:,3]                           # CHANGED FROM y1 = df_scaled.iloc[:,0] for categorical
            if data == 1 or data == 2:
                groups1 = groups2 = df_scaled['famid']
            elif data == 3 or data == 4:
                groups1 = groups2 = df_scaled['fid']
            else:
                print('Dataset not set')

            X2 = df_scaled.iloc[:,X2_vars]
            y2 = df.iloc[:,3]                           # CHANGED FROM y1 = df_scaled.iloc[:,0] for categorical
            features_full1 = ['Age']
            for i in [X2_vars[1:]]:
                features_full1.extend(df_scaled.columns[i])
            features_demo = features_full1[0:3]

            if data == 3 or data == 4:
                X3 = df_scaled_i.iloc[:,X3_vars]
                y3 = df_with_i.iloc[:,3]                # CHANGED FROM y1 = df_scaled.iloc[:,0] for categorical
                groups3 = df_scaled_i['fid']            # CHANGED FROM df_scaled.columns TO GET CORRECT # FAMID/GROUPS
                features_full2 = ['Age']
                for i in [X3_vars[1:]]:
                    features_full2.extend(df_scaled_i.columns[i])


            ##############################
            # CROSS-VALIDATION SELECTION #
            ##############################

            inner_rstate = 9052017
            outer_rstate = 12301985
            #inner_rstate = np.random.RandomState(9052017)
            #outer_rstate = np.random.RandomState(12301985)

            cv=GroupShuffleSplit
            #cv=GroupKFold
            cv_inner = cv(n_splits=20, test_size=0.2, random_state=inner_rstate)
            cv_outer = cv(n_splits=10, test_size=0.2, random_state=outer_rstate)
            n_iter=200
            n_iter_str = str(n_iter)

            print("Features Selected")


            #########################################
            ### RESULTS WITH DEMOGRAPHIC FEATURES ###
            #########################################

            for train1, test1 in cv_inner.split(X1, y1, groups=groups1):
                X_train1, X_test1 = X1.values[train1], X1.values[test1]
                y_train1, y_test1 = y1[train1], y1[test1]
                groups_train1, groups_test1 = groups1[train1], groups1[test1]

            search_time_start1 = time.time()
            if estimator == 5:
                clf1 = alg
            elif estimator == 4:
                spw1 = (len(y1)-np.sum(y1))/(np.sum(y1))
                alg1 = XGBClassifier(scale_pos_weight=spw1)
                clf1 = RandomizedSearchCV(alg1, param_dist, n_iter=n_iter, iid=False, cv=cv_inner, n_jobs=4)
                clf1.fit(X_train1, y_train1, groups = groups_train1)
            else:
                clf1 = RandomizedSearchCV(alg, param_dist, n_iter=n_iter, iid=False, cv=cv_inner, n_jobs=4)
                clf1.fit(X_train1, y_train1, groups = groups_train1)

            search_time_stop1 = time.time()

            if estimator == 5:
                cvalscorer1 = clf1
            else:
                cvalscorer1 = clf1.best_estimator_

            train1_scores = cross_val_score(cvalscorer1, X_train1, y_train1, groups=groups_train1, cv=cv_inner, n_jobs=4)
            test1_scores = cross_val_score(cvalscorer1, X_test1, y_test1, groups=groups_test1, cv=cv_outer, n_jobs=4)

            #################################
            ### RESULTS WITH ALL FEATURES ###
            #################################

            for train2, test2 in cv_inner.split(X2, y2, groups=groups2):
                X_train2, X_test2 = X2.values[train2], X2.values[test2]
                y_train2, y_test2 = y2[train2], y2[test2]
                groups_train2, groups_test2 = groups2[train2], groups2[test2]

            search_time_start2 = time.time()
            if estimator == 5:
                clf2 = alg
            elif estimator == 4:
                spw2 = (len(y2)-np.sum(y2))/(np.sum(y2))
                alg2 = XGBClassifier(scale_pos_weight=spw2)
                clf2 = RandomizedSearchCV(alg2, param_dist, n_iter=n_iter, iid=False, cv=cv_inner, n_jobs=4)
                clf2.fit(X_train2, y_train2, groups = groups_train2)
            else:
                clf2 = RandomizedSearchCV(alg, param_dist, n_iter=n_iter, iid=False, cv=cv_inner, n_jobs=4)
                clf2.fit(X_train2, y_train2, groups = groups_train2)

            search_time_stop2 = time.time()

            if estimator == 5:
                cvalscorer2 = clf2
            else:
                cvalscorer2 = clf2.best_estimator_

            train2_scores = cross_val_score(cvalscorer2, X_train2, y_train2, groups=groups_train2, cv=cv_inner, n_jobs=4)
            test2_scores = cross_val_score(cvalscorer2, X_test2, y_test2, groups=groups_test2, cv=cv_outer, n_jobs=4)

            if estimator != 5:
                print("Best Demo Estimator:\n", clf1.best_estimator_)
                print("")
                print("Best Demo Model params:")
                best_params1 = clf1.best_params_
                for param_name in sorted(best_params1.keys()):
                    print('%s: %r' % (param_name, best_params1[param_name]))

            if estimator != 5:
                print("Best Full Estimator:\n", clf2.best_estimator_)
                print("")
                print("Best Full params:")
                best_params2 = clf2.best_params_
                for param_name in sorted(best_params2.keys()):
                    print('%s: %r' % (param_name, best_params2[param_name]))

            ###########################################
            ### RESULTS WITH ALL FEATURES & IMAGING ###
            ###########################################

            if data == 3 or data == 4:

                for train3, test3 in cv_inner.split(X3, y3, groups=groups3):
                    X_train3, X_test3 = X3.values[train3], X3.values[test3]
                    y_train3, y_test3 = y3[train3], y3[test3]
                    groups_train3, groups_test3 = groups3[train3], groups3[test3]

                search_time_start3 = time.time()
                if estimator == 5:
                    clf3 = alg
                elif estimator == 4:
                    spw3 = (len(y3)-np.sum(y3))/(np.sum(y3))
                    alg3 = XGBClassifier(scale_pos_weight=spw3)
                    clf3 = RandomizedSearchCV(alg3, param_dist, n_iter=n_iter, iid=False, cv=cv_inner, n_jobs=4)
                    clf3.fit(X_train3, y_train3, groups = groups_train3)
                else:
                    clf3 = RandomizedSearchCV(alg, param_dist, n_iter=n_iter, iid=False, cv=cv_inner, n_jobs=4)
                    clf3.fit(X_train3, y_train3, groups = groups_train3)
                search_time_stop3 = time.time()

                if estimator == 5:
                    cvalscorer3 = clf3
                else:
                    cvalscorer3 = clf3.best_estimator_

                train3_scores = cross_val_score(cvalscorer3, X_train3, y_train3, groups=groups_train3, cv=cv_inner, n_jobs=4)
                test3_scores = cross_val_score(cvalscorer3, X_test3, y_test3, groups=groups_test3, cv=cv_outer, n_jobs=4)

                if estimator != 5:
                    print("Best Full Estimator:\n", clf3.best_estimator_)
                    print("")
                    print("Best Full params:")
                    best_params3 = clf3.best_params_
                    for param_name in sorted(best_params3.keys()):
                        print('%s: %r' % (param_name, best_params3[param_name]))
            else:
                print('No Imaging Data in CLDRC')


            #################################
            ###   FULL CONTENT TO PRINT   ###
            #################################

            print("Dataset_Name: ", dataset_name)
            print("Model_Variable: ", model_var)
            print("Model_Estimator: ", model_estimator)
            print("Imaging_File: ", i_filename)
            print("")
            print("Data_Preprocessed_-_Scaler_Applied")
            print("Features_Selected")
            print("Inner_Training_CV: ", cv_inner)
            print("Outer_Testing_CV: ", cv_outer)
            print("")

            print("Model_Tuning_Times: \n")
            print("Demographics_tuning_time: ")
            print("%.3f" %(search_time_stop1 - search_time_start1))
            print("")
            print("Full_Model_tuning_time: ")
            print("%.3f" %(search_time_stop2 - search_time_start2))
            print("")

            if data == 3 or data == 4:
                print("Full_Model_&_Imaging_tuning_time: ")
                print("%.3f" %(search_time_stop3 - search_time_start3))
            print("")

            print("DEMOGRAPHIC_MODEL")
            print("")
            if estimator == 5:
                print("No Best Estimator for LinearRegression")
            else:
                print("Demo_Best_Estimator:\n", clf1.best_estimator_)
            print("")
            print("Demo_Train_R2s_Mean:\n%.3f" %train1_scores.mean())
            print("Demo_Train_R2s_SD:\n%.3f" %train1_scores.std())
            print("")
            print("Demo_Test_R2s_Mean:\n%.3f" %test1_scores.mean())
            print("Demo_Test_R2s_SD:\n%.3f" %test1_scores.std())
            print("")

            print("FULL_MODEL")
            print("")
            if estimator == 5:
                print("No Best Estimator for LinearRegression")
            else:
                print("Full_Best_Estimator:\n", clf2.best_estimator_)
            print("")
            print("Full_Train_R2s_Mean:\n%.3f" %train2_scores.mean())
            print("Full_Train_R2s_SD:\n%.3f" %train2_scores.std())
            print("")
            print("Full_Test_R2s_Mean:\n%.3f" %test2_scores.mean())
            print("Full_Test_R2s_SD:\n%.3f" %test2_scores.std())
            print("")

            if data == 3 or data == 4:
                print("FULL_&_IMAGING_MODEL")
                print("")
                if estimator == 5:
                    print("No Best Estimator for LinearRegression")
                else:
                    print("Imaging_Best_Estimator:\n", clf3.best_estimator_)
                print("")
                print("Imaging_Train_R2s_Mean:\n%.3f" %train3_scores.mean())
                print("Imaging_Train_R2s_SD:\n%.3f" %train3_scores.std())
                print("")
                print("Imaging_Test_R2s_Mean:\n%.3f" %test3_scores.mean())
                print("Imaging_Test_R2s_SD:\n%.3f" %test3_scores.std())
                print("")


            ############################
            #### DIAGNOSTIC ACCURACY ###
            ############################

            cvreg=StratifiedShuffleSplit
            cvreg_inner = cvreg(n_splits=20, test_size=0.333, random_state=123085)
            cvreg_outer = cvreg(n_splits=5, test_size=0.333, random_state=90517)

            if estimator == 5:
                cvalscorer1 = clf1
                cvalscorer2 = clf2
                if data == 3 or data == 4:
                    cvalscorer3 = clf3
            else:
                cvalscorer1 = clf1.best_estimator_
                cvalscorer2 = clf2.best_estimator_
                if data == 3 or data == 4:
                    cvalscorer3 = clf3.best_estimator_     # CURRENT ERROR POINT #

            cm1_score=[]
            cm1_truepos=[]
            cm1_falseneg=[]
            cm1_falsepos=[]
            cm1_trueneg=[]
            cm1f1_weight=[]
            sensitivity1_avg=[]

            cm2_score=[]
            cm2_truepos=[]
            cm2_falseneg=[]
            cm2_falsepos=[]
            cm2_trueneg=[]
            cm2f1_weight=[]
            sensitivity2_avg=[]

            if data == 3 or data == 4:
                cm3_score=[]
                cm3_truepos=[]
                cm3_falseneg=[]
                cm3_falsepos=[]
                cm3_trueneg=[]
                cm3f1_weight=[]
                sensitivity3_avg=[]

            print(dataset_name, model_estimator, model_var, "Diagnostic Accuracy")

            print('')
            print('DEMOGRAPHICS MODEL ACCURACY')
            for k, (trainCV, testCV) in enumerate(cvreg_outer.split(X1, y1, groups=groups1)):
                cvalscorer1.fit(X1.values[trainCV], y1[trainCV])
                y1_pred = cvalscorer1.predict(X1.values[testCV])
                cm1 = confusion_matrix(y1[testCV],y1_pred, labels=[1,0])
                accuracy1=(cm1[0,0]+cm1[1,1])/sum(sum(cm1))
                sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
                specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
                if variable == 5:
                    print("[fold {0}] score: {1:.4f}, N_dx: {2}, truepos: {3}, trueneg: {4}, acc: {5:.4f}, sens: {6:.4f}, spec: {7:.4f}, f1_avg0: {8:.4f}, f1_avg1: {9:.4f}, f1_avg2: {10:.4f}, f1_avg3: {11:.4f}, f1_weight: {12:.4f}".
                          format(k, cvalscorer1.score(X1.values[testCV], y1[testCV]), sum(y1[testCV]), cm1[0,0], cm1[1,1], accuracy1, sensitivity1, specificity1, f1_score(y1[testCV],y1_pred,average=None)[0], f1_score(y1[testCV],y1_pred,average=None)[1], f1_score(y1[testCV],y1_pred,average=None)[2], f1_score(y1[testCV],y1_pred,average=None)[3], f1_score(y1[testCV],y1_pred,average='weighted')))
                else:
                    print("[fold {0}] score: {1:.4f}, N_dx: {2}, truepos: {3}, trueneg: {4}, acc: {5:.4f}, sens: {6:.4f}, spec: {7:.4f}, f1_avg: {8:.4f}, f1_weight: {9:.4f}".
                          format(k, cvalscorer1.score(X1.values[testCV], y1[testCV]), sum(y1[testCV]), cm1[0,0], cm1[1,1], accuracy1, sensitivity1, specificity1, f1_score(y1[testCV],y1_pred,average='binary'), f1_score(y1[testCV],y1_pred,average='weighted')))
                cm1_score.append(cvalscorer1.score(X1.values[testCV], y1[testCV]))
                cm1_truepos.append(cm1[0,0])
                cm1_falseneg.append(cm1[0,1])
                cm1_falsepos.append(cm1[1,0])
                cm1_trueneg.append(cm1[1,1])
                cm1f1_weight.append(f1_score(y1[testCV],y1_pred,average='weighted'))
                sensitivity1_avg.append(sensitivity1)

            print('')
            cm1_avg = [[np.mean(cm1_truepos),np.mean(cm1_falseneg)],[np.mean(cm1_falsepos),np.mean(cm1_trueneg)]]
            print(cm1_avg[0])
            print(cm1_avg[1])
            print('Demo Average Accuracy: ',np.mean(cm1_score))
            print('Demo Weighted F1 M: ',np.mean(cm1f1_weight))
            print('Demo Weighted F1 SD: ',np.std(cm1f1_weight))
            print('Demo Sensitivity M: ',np.mean(sensitivity1_avg))
            print('Demo Sensitivity SD: ',np.std(sensitivity1_avg))
            #print('Demo Sensitivity: ',np.mean(sensitivity2_avg))
#            print('Demo Precision: ',cm1_avg[0,0]/(cm1_avg[0,0]+cm1_avg[1,0]))
#            print('Demo Recall: ',cm1_avg[0,0]/(cm1_avg[0,0]+cm1_avg[0,1]))

            print('\n'*1)
            print('FULL MODEL ACCURACY')
            for k, (trainCV, testCV) in enumerate(cvreg_outer.split(X2, y2, groups=groups2)):
                cvalscorer2.fit(X2.values[trainCV], y2[trainCV])
                y2_pred = cvalscorer2.predict(X2.values[testCV])
                cm2 = confusion_matrix(y2[testCV],y2_pred, labels=[1,0])
                accuracy2=(cm2[0,0]+cm2[1,1])/sum(sum(cm2))
                sensitivity2 = cm2[0,0]/(cm2[0,0]+cm2[0,1])
                specificity2 = cm2[1,1]/(cm2[1,0]+cm2[1,1])
                if variable == 5:
                    print("[fold {0}] score: {1:.4f}, N_dx: {2}, truepos: {3}, trueneg: {4}, acc: {5:.4f}, sens: {6:.4f}, spec: {7:.4f}, f1_avg0: {8:.4f}, f1_avg1: {9:.4f}, f1_avg2: {10:.4f}, f1_avg3: {11:.4f}, f1_weight: {12:.4f}".
                          format(k, cvalscorer2.score(X2.values[testCV], y2[testCV]), sum(y2[testCV]), cm2[0,0], cm2[1,1], accuracy2, sensitivity2, specificity2, f1_score(y2[testCV],y2_pred,average=None)[0], f1_score(y2[testCV],y2_pred,average=None)[1], f1_score(y2[testCV],y2_pred,average=None)[2], f1_score(y2[testCV],y2_pred,average=None)[3], f1_score(y2[testCV],y2_pred,average='weighted')))
                else:
                    print("[fold {0}] score: {1:.4f}, N_dx: {2}, truepos: {3}, trueneg: {4}, acc: {5:.4f}, sens: {6:.4f}, spec: {7:.4f}, f1_avg: {8:.4f}, f1_weight: {9:.4f}".
                          format(k, cvalscorer2.score(X2.values[testCV], y2[testCV]), sum(y2[testCV]), cm2[0,0], cm2[1,1], accuracy2, sensitivity2, specificity2, f1_score(y2[testCV],y2_pred,average='binary'), f1_score(y2[testCV],y2_pred,average='weighted')))
                cm2_score.append(cvalscorer2.score(X2.values[testCV], y2[testCV]))
                cm2_truepos.append(cm2[0,0])
                cm2_falseneg.append(cm2[0,1])
                cm2_falsepos.append(cm2[1,0])
                cm2_trueneg.append(cm2[1,1])
                cm2f1_weight.append(f1_score(y2[testCV],y2_pred,average='weighted'))
                sensitivity2_avg.append(sensitivity2)

            print('')
            cm2_avg = [[np.mean(cm2_truepos),np.mean(cm2_falseneg)],[np.mean(cm2_falsepos),np.mean(cm2_trueneg)]]
            print(cm2_avg[0])
            print(cm2_avg[1])
            print('Full Average Accuracy: ',np.mean(cm2_score))
            print('Full Weighted F1 M: ',np.mean(cm2f1_weight))
            print('Full Weighted F1 SD: ',np.std(cm2f1_weight))
            print('Full Sensitivity M: ',np.mean(sensitivity2_avg))
            print('Full Sensitivity SD: ',np.std(sensitivity2_avg))
#            print('Full Precision: ',cm2_avg[0,0]/(cm2_avg[0,0]+cm2_avg[1,0]))
#            print('Full Recall: ',cm2_avg[0,0]/(cm2_avg[0,0]+cm2_avg[0,1]))

            if data == 3 or data == 4:
                print('\n'*1)
                print('FULL + IMAGING MODEL ACCURACY')
                for k, (trainCV, testCV) in enumerate(cvreg_outer.split(X3, y3, groups=groups3)):
                    cvalscorer3.fit(X3.values[trainCV], y3[trainCV])
                    y3_pred = cvalscorer3.predict(X3.values[testCV])
                    cm3 = confusion_matrix(y3[testCV],y3_pred, labels=[1,0])
                    accuracy3=(cm3[0,0]+cm3[1,1])/sum(sum(cm3))
                    sensitivity3 = cm3[0,0]/(cm3[0,0]+cm3[0,1])
                    specificity3 = cm3[1,1]/(cm3[1,0]+cm3[1,1])
                    if variable == 5:
                        print("[fold {0}] score: {1:.4f}, N_dx: {2}, truepos: {3}, trueneg: {4}, acc: {5:.4f}, sens: {6:.4f}, spec: {7:.4f}, f1_avg0: {8:.4f}, f1_avg1: {9:.4f}, f1_avg2: {10:.4f}, f1_avg3: {11:.4f}, f1_weight: {12:.4f}".
                              format(k, cvalscorer3.score(X3.values[testCV], y3[testCV]), sum(y3[testCV]), cm3[0,0], cm3[1,1], accuracy3, sensitivity3, specificity3, f1_score(y3[testCV],y3_pred,average=None)[0], f1_score(y3[testCV],y3_pred,average=None)[1], f1_score(y3[testCV],y3_pred,average=None)[2], f1_score(y3[testCV],y3_pred,average=None)[3], f1_score(y3[testCV],y3_pred,average='weighted')))
                    else:
                        print("[fold {0}] score: {1:.4f}, N_dx: {2}, truepos: {3}, trueneg: {4}, acc: {5:.4f}, sens: {6:.4f}, spec: {7:.4f}, f1_avg: {8:.4f}, f1_weight: {9:.4f}".
                              format(k, cvalscorer3.score(X3.values[testCV], y3[testCV]), sum(y3[testCV]), cm3[0,0], cm3[1,1], accuracy3, sensitivity3, specificity3, f1_score(y3[testCV],y3_pred,average='binary'), f1_score(y3[testCV],y3_pred,average='weighted')))
                    cm3_score.append(cvalscorer3.score(X3.values[testCV], y3[testCV]))
                    cm3_truepos.append(cm3[0,0])
                    cm3_falseneg.append(cm3[0,1])
                    cm3_falsepos.append(cm3[1,0])
                    cm3_trueneg.append(cm3[1,1])
                    cm3f1_weight.append(f1_score(y3[testCV],y3_pred,average='weighted'))
                    sensitivity3_avg.append(sensitivity3)

                print('')
                cm3_avg = [[np.mean(cm3_truepos),np.mean(cm3_falseneg)],[np.mean(cm3_falsepos),np.mean(cm3_trueneg)]]
                print(cm3_avg[0])
                print(cm3_avg[1])
                print('Imaging Average Accuracy: ',np.mean(cm3_score))
                print('Imaging Weighted F1 M: ',np.mean(cm3f1_weight))
                print('Imaging Weighted F1 SD: ',np.std(cm3f1_weight))
                print('Imaging Sensitivity M: ',np.mean(sensitivity3_avg))
                print('Imaging Sensitivity SD: ',np.std(sensitivity3_avg))
#                print('Imaging Precision: ',cm3_avg[0,0]/(cm3_avg[0,0]+cm3_avg[1,0]))
#                print('Imaging Recall: ',cm3_avg[0,0]/(cm3_avg[0,0]+cm3_avg[0,1]))
            else:
                print('Imaging not in CLDRC')
            print('')
            print('')
            print('')
            print('')
            print('')

In [ ]:
if variable == 1:
    with open("0.results_dxaccuracy_estimators_READ_f1recall_noHubDropNA.txt", 'w') as f:
        f.write(capture.stdout)
elif variable == 2:
    with open("0.results_dxaccuracy_estimators_IN_f1recall_noHubDropNA.txt", 'w') as f:
        f.write(capture.stdout)
elif variable == 3:
    with open("0.results_dxaccuracy_estimators_HI_f1recall_noHubDropNA.txt", 'w') as f:
        f.write(capture.stdout)
elif variable == 4:
    with open("0.results_dxaccuracy_estimators_ADHD_f1recall_noHubDropNA.txt", 'w') as f:
        f.write(capture.stdout)
elif variable == 5:
    with open("0.results_dxaccuracy_estimators_ADHDTYPE_f1recall_noHubDropNA.txt", 'w') as f:
        f.write(capture.stdout)
else:
    print('Variable not correctly set')

In [ ]:
# ORIGINAL DIAGNOSTIC ACCURACY CODE

############################
#### DIAGNOSTIC ACCURACY ###
############################

cvreg=StratifiedShuffleSplit
cvreg_inner = cvreg(n_splits=20, test_size=0.333, random_state=123085)
cvreg_outer = cvreg(n_splits=5, test_size=0.333, random_state=90517)

if estimator == 5:
    cvalscorer1 = clf1
    cvalscorer2 = clf2
    cvalscorer3 = clf3
else:
    cvalscorer1 = clf1.best_estimator_
    cvalscorer2 = clf2.best_estimator_
    cvalscorer3 = clf3.best_estimator_

cm1_score=[]
cm1_truepos=[]
cm1_falseneg=[]
cm1_falsepos=[]
cm1_trueneg=[]
cm1f1_weight=[]

cm2_score=[]
cm2_truepos=[]
cm2_falseneg=[]
cm2_falsepos=[]
cm2_trueneg=[]
cm2f1_weight=[]

cm3_score=[]
cm3_truepos=[]
cm3_falseneg=[]
cm3_falsepos=[]
cm3_trueneg=[]
cm3f1_weight=[]

print(dataset_name, model_estimator, model_var, "Diagostic Accuracy")

print('')
print('DEMOGRAPHICS MODEL ACCURACY')
for k, (trainCV, testCV) in enumerate(cvreg_outer.split(X1, y1, groups=groups1)):
    cvalscorer1.fit(X1.values[trainCV], y1[trainCV])
    y1_pred = cvalscorer1.predict(X1.values[testCV])
    cm1 = confusion_matrix(y1[testCV],y1_pred, labels=[1,0])
    accuracy1=(cm1[0,0]+cm1[1,1])/sum(sum(cm1))
    sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    print("[fold {0}] score: {1:.4f}, N_dx: {2}, truepos: {3}, trueneg: {4}, acc: {5:.4f}, sens: {6:.4f}, spec: {7:.4f}, f1_avg {8:.4f}, f1_weight {9:.4f}".
          format(k, cvalscorer1.score(X1.values[testCV], y1[testCV]), sum(y1[testCV]), cm1[0,0], cm1[1,1], accuracy1, sensitivity1, specificity1, f1_score(y1[testCV],y1_pred,average='binary'), f1_score(y1[testCV],y1_pred,average='weighted')))
    cm1_score.append(cvalscorer1.score(X1.values[testCV], y1[testCV]))
    cm1_truepos.append(cm1[0,0])
    cm1_falseneg.append(cm1[0,1])
    cm1_falsepos.append(cm1[1,0])
    cm1_trueneg.append(cm1[1,1])
    cm1f1_weight.append(f1_score(y1[testCV],y1_pred,average='weighted'))

print('')
cm1_avg = [[np.mean(cm1_truepos),np.mean(cm1_falseneg)],[np.mean(cm1_falsepos),np.mean(cm1_trueneg)]]
print(cm1_avg[0])
print(cm1_avg[1])
print('Demo Average Accuracy: ',np.mean(cm1_score))
print('Demo Weighted F1: ',np.mean(cm1f1_weight))

print('\n'*1)
print('FULL MODEL ACCURACY')
for k, (trainCV, testCV) in enumerate(cvreg_outer.split(X2, y2, groups=groups2)):
    cvalscorer2.fit(X2.values[trainCV], y2[trainCV])
    y2_pred = cvalscorer2.predict(X2.values[testCV])
    cm2 = confusion_matrix(y2[testCV],y2_pred, labels=[1,0])
    accuracy2=(cm2[0,0]+cm2[1,1])/sum(sum(cm2))
    sensitivity2 = cm2[0,0]/(cm2[0,0]+cm2[0,1])
    specificity2 = cm2[1,1]/(cm2[1,0]+cm2[1,1])
    print("[fold {0}] score: {1:.4f}, N_dx: {2}, truepos: {3}, trueneg: {4}, acc: {5:.4f}, sens: {6:.4f}, spec: {7:.4f}, f1_avg {8:.4f}, f1_weight {9:.4f}".
          format(k, cvalscorer2.score(X2.values[testCV], y2[testCV]), sum(y2[testCV]), cm2[0,0], cm2[1,1], accuracy2, sensitivity2, specificity2, f1_score(y2[testCV],y2_pred,average='binary'), f1_score(y2[testCV],y2_pred,average='weighted')))
    cm2_score.append(cvalscorer2.score(X2.values[testCV], y2[testCV]))
    cm2_truepos.append(cm2[0,0])
    cm2_falseneg.append(cm2[0,1])
    cm2_falsepos.append(cm2[1,0])
    cm2_trueneg.append(cm2[1,1])
    cm2f1_weight.append(f1_score(y2[testCV],y2_pred,average='weighted'))

print('')
cm2_avg = [[np.mean(cm2_truepos),np.mean(cm2_falseneg)],[np.mean(cm2_falsepos),np.mean(cm2_trueneg)]]
print(cm2_avg[0])
print(cm2_avg[1])
print('Full Average Accuracy: ',np.mean(cm2_score))
print('Full Weighted F1: ',np.mean(cm2f1_weight))

print('\n'*1)
print('FULL + IMAGING MODEL ACCURACY')
for k, (trainCV, testCV) in enumerate(cvreg_outer.split(X3, y3, groups=groups3)):
    cvalscorer3.fit(X3.values[trainCV], y3[trainCV])
    y3_pred = cvalscorer3.predict(X3.values[testCV])
    cm3 = confusion_matrix(y3[testCV],y3_pred, labels=[1,0])
    accuracy3=(cm3[0,0]+cm3[1,1])/sum(sum(cm3))
    sensitivity3 = cm3[0,0]/(cm3[0,0]+cm3[0,1])
    specificity3 = cm3[1,1]/(cm3[1,0]+cm3[1,1])
    print("[fold {0}] score: {1:.4f}, N_dx: {2}, truepos: {3}, trueneg: {4}, acc: {5:.4f}, sens: {6:.4f}, spec: {7:.4f}, f1_avg {8:.4f}, f1_weight {9:.4f}".
          format(k, cvalscorer3.score(X3.values[testCV], y3[testCV]), sum(y3[testCV]), cm3[0,0], cm3[1,1], accuracy3, sensitivity3, specificity3, f1_score(y3[testCV],y3_pred,average='binary'), f1_score(y3[testCV],y3_pred,average='weighted')))
    cm3_score.append(cvalscorer3.score(X3.values[testCV], y3[testCV]))
    cm3_truepos.append(cm3[0,0])
    cm3_falseneg.append(cm3[0,1])
    cm3_falsepos.append(cm3[1,0])
    cm3_trueneg.append(cm3[1,1])
    cm3f1_weight.append(f1_score(y3[testCV],y3_pred,average='weighted'))

print('')
cm3_avg = [[np.mean(cm3_truepos),np.mean(cm3_falseneg)],[np.mean(cm3_falsepos),np.mean(cm3_trueneg)]]
print(cm3_avg[0])
print(cm3_avg[1])
print('Imaging Average Accuracy: ',np.mean(cm3_score))
print('Imaging Weighted F1: ',np.mean(cm3f1_weight))

In [ ]:
with open("0.results_dxaccuracy_estimators_READ.txt", 'w') as f:
    f.write(capture.stdout)

#with open("0.results_dxaccuracy_estimators_IN.txt", 'w') as f:
#    f.write(capture.stdout)

#with open("0.results_dxaccuracy_estimators_HI.txt", 'w') as f:
#    f.write(capture.stdout)

In [ ]:
#########################################
### RESULTS WITH DEMOGRAPHIC FEATURES ###
#########################################

print("Dataset: ", dataset_name)
print("Model Variable: ", model_var)
print("Model Estimator: ", model_estimator)
print("")

#print("Model Tuning Times: \n")

#for train1, test1 in cv_inner.split(X1, y1, groups=groups1):
#    X_train1, X_test1 = X1.values[train1], X1.values[test1]
#    y_train1, y_test1 = y1[train1], y1[test1]
#    groups_train1, groups_test1 = groups1[train1], groups1[test1]

#search_time_start1 = time.time()
#print("Demographics tuning time...")
#if estimator == 5:
#    clf1 = alg
#else:
#    clf1 = RandomizedSearchCV(alg, param_dist, n_iter=n_iter, iid=False, cv=cv_inner, n_jobs=4)
#    clf1.fit(X_train1, y_train1, groups = groups_train1)

#search_time_stop1 = time.time()
#print('%.3f' %(search_time_stop1 - search_time_start1))
#print("")

#if estimator == 5:
#    cvalscorer1 = clf1
#else:
#    cvalscorer1 = clf1.best_estimator_

#train1_scores = cross_val_score(cvalscorer1, X_train1, y_train1, groups=groups_train1, cv=cv_inner, n_jobs=4)
#test1_scores = cross_val_score(cvalscorer1, X_test1, y_test1, groups=groups_test1, cv=cv_outer, n_jobs=4)

#################################
### RESULTS WITH ALL FEATURES ###
#################################

for train2, test2 in cv_inner.split(X2, y2, groups=groups2):
    X_train2, X_test2 = X2.values[train2], X2.values[test2]
    y_train2, y_test2 = y2[train2], y2[test2]
    groups_train2, groups_test2 = groups2[train2], groups2[test2]
    r_train2, r_test2 = r2[train2], r2[test2]
    i_train2, i_test2 = i2[train2], i2[test2]
    h_train2, h_test2 = h2[train2], h2[test2]
    m_train2, m_test2 = m2[train2], m2[test2]

#search_time_start2 = time.time()
#print("Full Model tuning time...")
#if estimator == 5:
#    clf2 = alg
#else:
#    clf2 = RandomizedSearchCV(alg, param_dist, n_iter=n_iter, iid=False, cv=cv_inner, n_jobs=4)
#    clf2.fit(X_train2, y_train2, groups = groups_train2)

#search_time_stop2 = time.time()
#print('%.3f' %(search_time_stop2 - search_time_start2))
#print("\n"*2)

#if estimator == 5:
#    cvalscorer2 = clf2
#else:
#    cvalscorer2 = clf2.best_estimator_

train2_scores = cross_val_score(lasso_cvb, X_train2, y_train2, groups=groups_train2, cv=cv_inner, n_jobs=4)
test2_scores = cross_val_score(lasso_cvb, X_test2, y_test2, groups=groups_test2, cv=cv_outer, n_jobs=4)

#print("DEMOGRAPHIC MODEL RESULTS")
#print("")
#print("Train [inner CV] R2s:\n", train1_scores)
#print("Train R2s Mean:\n%.3f" %train1_scores.mean())
#print("Train R2s SD:\n%.3f" %train1_scores.std())
#print("")
#print("Test [outer CV] R2s:\n", test1_scores)
#print("Test R2s Mean:\n%.3f" %test1_scores.mean())
#print("Test R2s SD:\n%.3f" %test1_scores.std())
#print("")
#if estimator != 5:
#    print("Best Demo Estimator:\n", clf1.best_estimator_)
#    print("")
#    print("Best Demo Model params:")
#    best_params1 = clf1.best_params_
#    for param_name in sorted(best_params1.keys()):
#        print('%s: %r' % (param_name, best_params1[param_name]))
print("\n"*2)
print("FULL BEHAVIORAL MODEL RESULTS")
print("")
print("Train [inner CV] R2s:\n", train2_scores)
print("Train R2s Mean:\n%.3f" %train2_scores.mean())
print("Train R2s SD:\n%.3f" %train2_scores.std())
print("")
print("Test [outer CV] R2s:\n", test2_scores)
print("Test R2s Mean:\n%.3f" %test2_scores.mean())
print("Test R2s SD:\n%.3f" %test2_scores.std())
print("")
#if estimator != 5:
#    print("Best Full Estimator:\n", clf2.best_estimator_)
#    print("")
#    print("Best Full params:")
#    best_params2 = clf2.best_params_
#    for param_name in sorted(best_params2.keys()):
#        print('%s: %r' % (param_name, best_params2[param_name]))

# Feature Weights

In [ ]:
print("MODEL FEATURE WEIGHTS")

print("Dataset: ", dataset_name)
print("Model Variable: ", model_var)
print("Model Estimator: ", model_estimator)
print("\n"*2)

if estimator == 1:
    alg1_coef = LinearSVR(max_iter = 10000, C = clf1.best_params_['C'], epsilon = clf1.best_params_['epsilon'])
    alg1_for_coef = alg1_coef.fit(X_train1, y_train1)
    alg2_coef = LinearSVR(max_iter = 10000, C = clf2.best_params_['C'], epsilon = clf2.best_params_['epsilon'])
    alg2_for_coef = alg2_coef.fit(X_train2, y_train2)
    if data == 3 or data == 4:
        alg3_coef = LinearSVR(max_iter = 10000, C = clf3.best_params_['C'], epsilon = clf3.best_params_['epsilon'])
        alg3_for_coef = alg3_coef.fit(X_train3, y_train3)
elif estimator == 2:
    alg1_coef = SVR(kernel = 'linear', C = clf1.best_params_['C'], epsilon = clf1.best_params_['epsilon'])
    alg1_for_coef = alg1_coef.fit(X_train1, y_train1)
    alg2_coef = SVR(kernel = 'linear', C = clf2.best_params_['C'], epsilon = clf2.best_params_['epsilon'])
    alg2_for_coef = alg2_coef.fit(X_train2, y_train2)
    if data == 3 or data == 4:
        alg3_coef = SVR(kernel = 'linear', C = clf3.best_params_['C'], epsilon = clf3.best_params_['epsilon'])
        alg3_for_coef = alg3_coef.fit(X_train3, y_train3)
elif estimator == 3:
    alg1_coef = SVR(kernel = 'rbf', gamma = 'auto', C = clf1.best_params_['C'], epsilon = clf1.best_params_['epsilon'])
    alg1_for_coef = alg1_coef.fit(X_train1, y_train1)
    alg2_coef = SVR(kernel = 'rbf', gamma = 'auto', C = clf2.best_params_['C'], epsilon = clf2.best_params_['epsilon'])
    alg2_for_coef = alg2_coef.fit(X_train2, y_train2)
    if data == 3 or data == 4:
        alg3_coef = SVR(kernel = 'rbf', gamma = 'auto', C = clf3.best_params_['C'], epsilon = clf3.best_params_['epsilon'])
        alg3_for_coef = alg3_coef.fit(X_train3, y_train3)
elif estimator == 4:
    alg1_coef = XGBRegressor(learning_rate = clf1.best_params_['learning_rate'], gamma = clf1.best_params_['gamma'])
    alg1_coef = XGBRegressor(max_depth = clf1.best_params_['max_depth'], min_child_weight = clf1.best_params_['min_child_weight'])
    alg1_coef = XGBRegressor(subsample = clf1.best_params_['subsample'], colsample_bytree = clf1.best_params_['colsample_bytree'])
    alg1_coef = XGBRegressor(reg_lambda = clf1.best_params_['reg_lambda'], reg_alpha = clf1.best_params_['reg_alpha'])
    alg1_for_coef = alg1_coef.fit(X_train1, y_train1)
    alg2_coef = XGBRegressor(learning_rate = clf2.best_params_['learning_rate'], gamma = clf2.best_params_['gamma'])
    alg2_coef = XGBRegressor(max_depth = clf2.best_params_['max_depth'], min_child_weight = clf2.best_params_['min_child_weight'])
    alg2_coef = XGBRegressor(subsample = clf2.best_params_['subsample'], colsample_bytree = clf2.best_params_['colsample_bytree'])
    alg2_coef = XGBRegressor(reg_lambda = clf2.best_params_['reg_lambda'], reg_alpha = clf2.best_params_['reg_alpha'])
    alg2_for_coef = alg2_coef.fit(X_train2, y_train2)
    if data == 3 or data == 4:
        alg3_coef = XGBRegressor(learning_rate = clf3.best_params_['learning_rate'], gamma = clf3.best_params_['gamma'])
        alg3_coef = XGBRegressor(max_depth = clf3.best_params_['max_depth'], min_child_weight = clf3.best_params_['min_child_weight'])
        alg3_coef = XGBRegressor(subsample = clf3.best_params_['subsample'], colsample_bytree = clf3.best_params_['colsample_bytree'])
        alg3_coef = XGBRegressor(reg_lambda = clf3.best_params_['reg_lambda'], reg_alpha = clf3.best_params_['reg_alpha'])
        alg3_for_coef = alg3_coef.fit(X_train3, y_train3)
elif estimator == 5:
    alg1_for_coef = clf1.fit(X_train1, y_train1)
    alg2_for_coef = clf2.fit(X_train2, y_train2)
    if data == 3 or data == 4:
        alg3_for_coef = clf3.fit(X_train3, y_train3)
else:
    print("Estimator Not Selected")

feature_list_demo = []
feature_list_full = []
feature_list_imaging = []

for i in list(range(0,len(X1.columns))):
    feature_list_demo.append([alg1_for_coef.coef_[i], X1.columns[i]])
    df_features_demo = pd.DataFrame(data=feature_list_demo, columns=['coef','feature'])
df_features_demo.reindex(df_features_demo.coef.abs().sort_values(ascending=False).index)
df_features_demo_sum = df_features_demo[(df_features_demo['coef'] > .001) | (df_features_demo['coef'] < -.001)]
df_features_demo_sum.reindex(df_features_demo_sum.coef.abs().sort_values(ascending=False).index)

for i in list(range(0,len(X2.columns))):
    feature_list_full.append([alg2_for_coef.coef_[i], X2.columns[i]])
    df_features_full = pd.DataFrame(data=feature_list_full, columns=['coef','feature'])
df_features_full.reindex(df_features_full.coef.abs().sort_values(ascending=False).index)
df_features_full_sum = df_features_full[(df_features_full['coef'] > .001) | (df_features_full['coef'] < -.001)]
df_features_full_sum.reindex(df_features_full_sum.coef.abs().sort_values(ascending=False).index)

for i in list(range(0,len(X3.columns))):
    feature_list_imaging.append([alg3_for_coef.coef_[i], X3.columns[i]])
    df_features_imaging = pd.DataFrame(data=feature_list_imaging, columns=['coef','feature'])
df_features_imaging.reindex(df_features_imaging.coef.abs().sort_values(ascending=False).index)
df_features_imaging_sum = df_features_imaging[(df_features_imaging['coef'] > .001) | (df_features_imaging['coef'] < -.001)]
df_features_imaging_sum.reindex(df_features_imaging_sum.coef.abs().sort_values(ascending=False).index)

if estimator == 1 or estimator == 2:
    print("Demographic Feature Weights:\n", features_demo, "\n", alg1_for_coef.coef_)
    print("")
    print("All Feature Weights:\n", features_full1, "\n", alg2_for_coef.coef_)
else:
    print("Weights only available for Linear Kernels")
if (data == 3 or data == 4) and (estimator == 1 or estimator ==2):
    print("")
    print("Full & Imaging Feature Weights:\n", features_full2, "\n", alg3_for_coef.coef_)
else:
    print("Full model with imaging not available")

# FULL REGULARIZATION STREAM

In [ ]:
from sklearn.model_selection import ShuffleSplit

cvreg=ShuffleSplit
cvreg_inner = cvreg(n_splits=20, test_size=0.2, random_state=123085)
cvreg_outer = cvreg(n_splits=9, test_size=0.2, random_state=90517)

##########################
### RidgeCV Behavioral ###
##########################

from sklearn.linear_model import RidgeCV

alphas_ridgecvb = np.logspace(-3, 2.3, 50)
ridge_cvb = RidgeCV(alphas=alphas_ridgecvb, cv=cvreg_inner, normalize=False)
ridge_cvb_score = []

for k, (trainCV, testCV) in enumerate(cvreg_outer.split(X2, y2, groups=groups2)):
    ridge_cvb.fit(X2.values[trainCV], y2[trainCV])
    print("[fold {0}] alpha: {1:.5f}, score: {2:.5f}".
          format(k, ridge_cvb.alpha_, ridge_cvb.score(X2.values[testCV], y2[testCV])))
    ridge_cvb_score.append(ridge_cvb.score(X2.values[testCV], y2[testCV]))

print('RidgeCVB')
print('Mean: ', np.mean(ridge_cvb_score))
print('SD: ', np.std(ridge_cvb_score))

feature_list_ridgeb = []
for i in list(range(0,len(X2.columns))):
    feature_list_ridgeb.append([ridge_cvb.coef_[i], X2.columns[i]])
    df_featimp_ridgeb = pd.DataFrame(data=feature_list_ridgeb, columns=['coef','feature'])
df_featimp_ridgeb.reindex(df_featimp_ridgeb.coef.abs().sort_values(ascending=False).index)
df_featimp_sum_ridgeb = df_featimp_ridgeb[(df_featimp_ridgeb['coef'] > .001) | (df_featimp_ridgeb['coef'] < -.001)]
print(df_featimp_sum_ridgeb.reindex(df_featimp_sum_ridgeb.coef.abs().sort_values(ascending=False).index))

#######################
### RidgeCV Imaging ###
#######################

alphas_ridgecvi = np.logspace(-3, 3, 50)
ridge_cvi = RidgeCV(alphas=alphas_ridgecvi, cv=cvreg_inner, normalize=False)
ridge_cvi_score = []

for k, (trainCV, testCV) in enumerate(cvreg_outer.split(X3, y3, groups=groups3)):
    ridge_cvi.fit(X3.values[trainCV], y3[trainCV])
    print("[fold {0}] alpha: {1:.5f}, score: {2:.5f}".
          format(k, ridge_cvi.alpha_, ridge_cvi.score(X3.values[testCV], y3[testCV])))
    ridge_cvi_score.append(ridge_cvi.score(X3.values[testCV], y3[testCV]))

print('RidgeCVI')
print('Mean: ', np.mean(ridge_cvi_score))
print('SD: ', np.std(ridge_cvi_score))

feature_list_ridgei = []
for i in list(range(0,len(X3.columns))):
    feature_list_ridgei.append([ridge_cvi.coef_[i], X3.columns[i]])
    df_featimp_ridgei = pd.DataFrame(data=feature_list_ridgei, columns=['coef','feature'])
df_featimp_ridgei.reindex(df_featimp_ridgei.coef.abs().sort_values(ascending=False).index)
df_featimp_sum_ridgei = df_featimp_ridgei[(df_featimp_ridgei['coef'] > .001) | (df_featimp_ridgei['coef'] < -.001)]
print(df_featimp_sum_ridgei.reindex(df_featimp_sum_ridgei.coef.abs().sort_values(ascending=False).index))

##########################
### LassoCV Behavioral ###
##########################

from sklearn.linear_model import LassoCV

alphas_lassocvb = np.logspace(-4, -0.3, 100)
lasso_cvb = LassoCV(alphas=alphas_lassocvb, cv=cvreg_inner, max_iter=100000, tol=0.0001, n_jobs=4)
lasso_cvb_score = []

for k, (trainCV, testCV) in enumerate(cvreg_outer.split(X2, y2, groups=groups2)):
    lasso_cvb.fit(X2.values[trainCV], y2[trainCV])
    print("[fold {0}] alpha: {1:.5f}, score: {2:.5f}".
          format(k, lasso_cvb.alpha_, lasso_cvb.score(X2.values[testCV], y2[testCV])))
    lasso_cvb_score.append(lasso_cvb.score(X2.values[testCV], y2[testCV]))

print('LassoCVB')
print('Mean: ', np.mean(lasso_cvb_score))
print('SD: ', np.std(lasso_cvb_score))

feature_list_lassob = []
for i in list(range(0,len(X2.columns))):
    feature_list_lassob.append([lasso_cvb.coef_[i], X2.columns[i]])
    df_featimp_lassob = pd.DataFrame(data=feature_list_lassob, columns=['coef','feature'])
df_featimp_lassob.reindex(df_featimp_lassob.coef.abs().sort_values(ascending=False).index)
df_featimp_sum_lassob = df_featimp_lassob[(df_featimp_lassob['coef'] > .001) | (df_featimp_lassob['coef'] < -.001)]
print(df_featimp_sum_lassob.reindex(df_featimp_sum_lassob.coef.abs().sort_values(ascending=False).index))

#######################
### LassoCV Imaging ###
#######################

alphas_lassocvi = np.logspace(-4, -0.3, 100)
lasso_cvi = LassoCV(alphas=alphas_lassocvi, cv=cvreg_inner, max_iter=100000, tol=0.0001, n_jobs=4)
lasso_cvi_score = []

for k, (trainCV, testCV) in enumerate(cvreg_outer.split(X3, y3, groups=groups3)):
    lasso_cvi.fit(X3.values[trainCV], y3[trainCV])
    print("[fold {0}] alpha: {1:.5f}, score: {2:.5f}".
          format(k, lasso_cvi.alpha_, lasso_cvi.score(X3.values[testCV], y3[testCV])))
    lasso_cvi_score.append(lasso_cvi.score(X3.values[testCV], y3[testCV]))

print('LassoCVI')
print('Mean: ', np.mean(lasso_cvi_score))
print('SD: ', np.std(lasso_cvi_score))

feature_list_lassoi = []
for i in list(range(0,len(X3.columns))):
    feature_list_lassoi.append([lasso_cvi.coef_[i], X3.columns[i]])
    df_featimp_lassoi = pd.DataFrame(data=feature_list_lassoi, columns=['coef','feature'])
df_featimp_lassoi.reindex(df_featimp_lassoi.coef.abs().sort_values(ascending=False).index)
df_featimp_sum_lassoi = df_featimp_lassoi[(df_featimp_lassoi['coef'] > .001) | (df_featimp_lassoi['coef'] < -.001)]
print(df_featimp_sum_lassoi.reindex(df_featimp_sum_lassoi.coef.abs().sort_values(ascending=False).index))


##############################
### LassoLarsCV Behavioral ###
##############################

from sklearn.linear_model import LassoLarsCV

llars_cvb = LassoLarsCV(cv=cvreg_inner, max_iter=10000, n_jobs=4)
llars_cvb_score = []

for k, (trainCV, testCV) in enumerate(cvreg_outer.split(X2, y2, groups=groups2)):
    llars_cvb.fit(X2.values[trainCV], y2[trainCV])
    print("[fold {0}] alpha: {1:.5f}, score: {2:.5f}".
          format(k, llars_cvb.alpha_, llars_cvb.score(X2.values[testCV], y2[testCV])))
    llars_cvb_score.append(llars_cvb.score(X2.values[testCV], y2[testCV]))

print('LassoLars CVB')
print('Mean: ', np.mean(llars_cvb_score))
print('SD: ', np.std(llars_cvb_score))

feature_list_llb = []
for i in list(range(0,len(X2.columns))):
    feature_list_llb.append([llars_cvb.coef_[i], X2.columns[i]])
    df_featimp_llb = pd.DataFrame(data=feature_list_llb, columns=['coef','feature'])
df_featimp_llb.reindex(df_featimp_llb.coef.abs().sort_values(ascending=False).index)
df_featimp_sum_llb = df_featimp_llb[(df_featimp_llb['coef'] > .001) | (df_featimp_llb['coef'] < -.001)]
print(df_featimp_sum_llb.reindex(df_featimp_sum_llb.coef.abs().sort_values(ascending=False).index))

###########################
### LassoLarsCV Imaging ###
###########################

llars_cvi = LassoLarsCV(cv=cvreg_inner, max_iter=10000, n_jobs=4)
llars_cvi_score = []

for k, (trainCV, testCV) in enumerate(cvreg_outer.split(X3, y3, groups=groups3)):
    llars_cvi.fit(X3.values[trainCV], y3[trainCV])
    print("[fold {0}] alpha: {1:.5f}, score: {2:.5f}".
          format(k, llars_cvi.alpha_, llars_cvi.score(X3.values[testCV], y3[testCV])))
    llars_cvi_score.append(llars_cvi.score(X3.values[testCV], y3[testCV]))

print('LassoLarsCVI')
print('Mean: ', np.mean(llars_cvi_score))
print('SD: ', np.std(llars_cvi_score))

feature_list_lli = []
for i in list(range(0,len(X3.columns))):
    feature_list_lli.append([llars_cvi.coef_[i], X3.columns[i]])
    df_featimp_lli = pd.DataFrame(data=feature_list_lli, columns=['coef','feature'])
df_featimp_lli.reindex(df_featimp_lli.coef.abs().sort_values(ascending=False).index)
df_featimp_sum_lli = df_featimp_lli[(df_featimp_lli['coef'] > .001) | (df_featimp_lli['coef'] < -.001)]
print(df_featimp_sum_lli.reindex(df_featimp_sum_lli.coef.abs().sort_values(ascending=False).index))

###############################
### ElasticNetCV Behavioral ###
###############################

from sklearn.linear_model import ElasticNetCV

alphas_enetcvb = np.logspace(-3, -0.000001, 30)
l1s_enetcvb = np.logspace(-3,-0.000001,30)
enet_cvb = ElasticNetCV(alphas=alphas_enetcvb, l1_ratio=l1s_enetcvb, cv=cvreg_inner, max_iter=10000, normalize=False, n_jobs=4)
enet_cvb_score = []

for k, (trainCV, testCV) in enumerate(cvreg_outer.split(X2, y2, groups=groups2)):
    enet_cvb.fit(X2.values[trainCV], y2[trainCV])
    print("[fold {0}] alpha: {1:.5f}, l1_ratio: {2:.5f}, score: {3:.5f}".
          format(k, enet_cvb.alpha_, enet_cvb.l1_ratio_, enet_cvb.score(X2.values[testCV], y2[testCV])))
    enet_cvb_score.append(enet_cvb.score(X2.values[testCV], y2[testCV]))

print('ElasticNetCVB')
print('Mean: ', np.mean(enet_cvb_score))
print('SD: ', np.std(enet_cvb_score))

feature_list_enb = []
for i in list(range(0,len(X2.columns))):
    feature_list_enb.append([enet_cvb.coef_[i], X2.columns[i]])
    df_featimp_enb = pd.DataFrame(data=feature_list_enb, columns=['coef','feature'])
df_featimp_enb.reindex(df_featimp_enb.coef.abs().sort_values(ascending=False).index)
df_featimp_sum_enb = df_featimp_enb[(df_featimp_enb['coef'] > .001) | (df_featimp_enb['coef'] < -.001)]
print(df_featimp_sum_enb.reindex(df_featimp_sum_enb.coef.abs().sort_values(ascending=False).index))

############################
### ElasticNetCV Imaging ###
############################

cvreg_inner = cvreg(n_splits=30, test_size=0.2, random_state=12301985)
cvreg_outer = cvreg(n_splits=15, test_size=0.2, random_state=905017)

alphas_enetcvi = np.logspace(-1.5, -.00001, 30)
l1s_enetcvi = np.logspace(-1,-.00001,30)
enet_cvi = ElasticNetCV(alphas=alphas_enetcvi, l1_ratio=l1s_enetcvi, cv=cvreg_inner, max_iter=10000, normalize=False, n_jobs=4)
enet_cvi_score = []

for k, (trainCV, testCV) in enumerate(cvreg_outer.split(X3, y3, groups=groups3)):
    enet_cvi.fit(X3.values[trainCV], y3[trainCV])
    print("[fold {0}] alpha: {1:.5f}, l1_ratio: {2:.5f}, score: {3:.5f}".
          format(k, enet_cvi.alpha_, enet_cvi.l1_ratio_, enet_cvi.score(X3.values[testCV], y3[testCV])))
    enet_cvi_score.append(enet_cvi.score(X3.values[testCV], y3[testCV]))

print('ElasticNetCVI')
print('Mean: ', np.mean(enet_cvi_score))
print('SD: ', np.std(enet_cvi_score))

feature_list_eni = []
for i in list(range(0,len(X3.columns))):
    feature_list_eni.append([enet_cvi.coef_[i], X3.columns[i]])
    df_featimp_eni = pd.DataFrame(data=feature_list_eni, columns=['coef','feature'])
df_featimp_eni.reindex(df_featimp_eni.coef.abs().sort_values(ascending=False).index)
df_featimp_sum_eni = df_featimp_eni[(df_featimp_eni['coef'] > .001) | (df_featimp_eni['coef'] < -.001)]
print(df_featimp_sum_eni.reindex(df_featimp_sum_eni.coef.abs().sort_values(ascending=False).index))

##########################################
### Regularization Performance Summary ###
##########################################

print("RidgeCVB Mean: {0:.5f}; SD: {1:.5f}; alpha: {2:.5f}".
      format(np.mean(ridge_cvb_score), np.std(ridge_cvb_score), ridge_cvb.alpha_))
print("RidgeCVI Mean: {0:.5f}; SD: {1:.5f}; alpha: {2:.5f}".
      format(np.mean(ridge_cvi_score), np.std(ridge_cvi_score), ridge_cvi.alpha_))
print("LassoCVB Mean: {0:.5f}; SD: {1:.5f}; alpha: {2:.5f}".
      format(np.mean(lasso_cvb_score), np.std(lasso_cvb_score), lasso_cvb.alpha_))
print("LassoCVI Mean: {0:.5f}; SD: {1:.5f}; alpha: {2:.5f}".
      format(np.mean(lasso_cvi_score), np.std(lasso_cvi_score), lasso_cvi.alpha_))
print("LLarsCVB Mean: {0:.5f}; SD: {1:.5f}; alpha: {2:.5f}".
      format(np.mean(llars_cvb_score), np.std(llars_cvb_score), llars_cvb.alpha_))
print("LLarsCVI Mean: {0:.5f}; SD: {1:.5f}; alpha: {2:.5f}".
      format(np.mean(llars_cvi_score), np.std(llars_cvi_score), llars_cvi.alpha_))
print("E-NetCVB Mean: {0:.5f}; SD: {1:.5f}; alpha: {2:.5f}; l1_ratio: {3:.5f}".
      format(np.mean(enet_cvb_score), np.std(enet_cvb_score), enet_cvb.alpha_, enet_cvb.l1_ratio_))
print("E-NetCVI Mean: {0:.5f}; SD: {1:.5f}; alpha: {2:.5f}; l1_ratio: {3:.5f}".
      format(np.mean(enet_cvi_score), np.std(enet_cvi_score), enet_cvi.alpha_, enet_cvi.l1_ratio_))